# Workflow to compare two sets of DISP data

- Compare two sets of DISP data extracted and prepared using the `run2_prep_mintpy_opera.py` script within the Cal/Val repo https://github.com/OPERA-Cal-Val/calval-DISP. 

<br>
<b><I>Notebook to Compare sets of DISP data</I></b><br>
- Authors: Simran Sangha, Mary Grace Bato, December 2025

<div class="alert alert-warning">
All sequential sections NEED to be run in order.
</div>



## 0. Papermill Parameters


In [ ]:
# PARAMETERS (Papermill will override these)

frame_num = "08882"  # e.g., '08882'
parent_dir = "/u/duvel-d2/ssangha/DISP_work/forward_mode"

# Primary and comparison versions
version_num = "historic"
version_num_to_comp = "forward"  # or None

# This is the threshold used for the *second* set of plots.
# First set is always run with threshold_density = None.
threshold_density = 0.9  # default; papermill can override

bins = 20

# Random timeseries comparison configuration
list_lat_lon = [
    (29.7784, -95.7445),
    (29.9056, -95.6402),
    (29.8606, -95.3851),
    (29.6318, -95.5671),
    (29.6061, -95.0003),
]
list_lat_lon_wkt_output = None
pixel_radius = 3
random_seed = 0
timeseries_filename = "timeseries.h5"

ref_lalo = None


In [ ]:
# injected parameters
figures_dir = f"F{frame_num}_figures"


## 1. Imports (Global)


In [ ]:
# all imports
import base64
import warnings
import glob
import os
from datetime import datetime as dt, timedelta
from io import BytesIO
from pathlib import Path
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union

import h5py
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, Markdown, display
from mintpy.cli import view, diff
from mintpy.objects import timeseries
from mintpy.utils import readfile, writefile, utils as ut
from osgeo import gdal
gdal.UseExceptions()
from pyproj import Transformer
from scipy import stats as st



## 2. Table of Contents

- [0. Papermill Parameters](#0-papermill-parameters)
- [1. Imports (Global)](#1-imports-global)
- [2. Table of Contents](#2-table-of-contents)
- [3. Timeseries Density Analysis](#3-timeseries-density-analysis)
  - [3.1 Helper Functions](#31-helper-functions)
  - [3.2 Core Plotting Routines](#32-core-plotting-routines)
  - [3.3 Run Both Threshold Configurations](#33-run-both-threshold-configurations)
  - [3.4 Execution Cell](#34-execution-cell)
- [4. Visualizing Layers by Epoch](#4-visualizing-layers-by-epoch)
  - [4.0 Visualize Selected Timeseries Points](#40-visualize-selected-timeseries-points)
  - [4.1 Recommended Mask](#41-recommended-mask)
  - [4.2 Displacement](#42-displacement)
    - [4.2.i Timeseries Comparisons](#42i-timeseries-comparisons)
  - [4.3 Short Wavelength Displacement](#43-short-wavelength-displacement)
    - [4.3.i Timeseries Comparisons](#43i-timeseries-comparisons)
  - [4.4 Ionospheric Delay](#44-ionospheric-delay)
    - [4.4.i Timeseries Comparisons](#44i-timeseries-comparisons)
  - [4.5 Solid Earth Tide](#45-solid-earth-tide)
    - [4.5.i Timeseries Comparisons](#45i-timeseries-comparisons)
  - [4.6 Perpendicular Baseline](#46-perpendicular-baseline)
    - [4.6.i Timeseries Comparisons](#46i-timeseries-comparisons)
  - [4.7 Connected Component Labels](#47-connected-component-labels)
  - [4.8 Shape Counts](#48-shape-counts)
    - [4.8.i Timeseries Comparisons](#48i-timeseries-comparisons)
  - [4.9 Temporal Coherence](#49-temporal-coherence)
    - [4.9.i Timeseries Comparisons](#49i-timeseries-comparisons)
  - [4.10 Estimated Phase Quality](#410-estimated-phase-quality)
    - [4.10.i Timeseries Comparisons](#410i-timeseries-comparisons)
  - [4.11 Phase Similarity](#411-phase-similarity)
    - [4.11.i Timeseries Comparisons](#411i-timeseries-comparisons)
  - [4.12 Water Mask](#412-water-mask)
  - [4.13 Timeseries Inversion Residuals](#413-timeseries-inversion-residuals)
    - [4.13.i Timeseries Comparisons](#413i-timeseries-comparisons)
  - [4.14 DEM Error](#414-dem-error)
- [5. Velocity Layers](#5-velocity-layers)
  - [5.1 Raw Velocity](#51-raw-velocity)
  - [5.2 DEM Error Corrected Velocity](#52-dem-error-corrected-velocity)
  - [5.3 Short Wavelength Velocity](#53-short-wavelength-velocity)
- [6. Range Summary](#6-range-summary)

---


## 3. Timeseries Density Analysis


### 3.1 Helper Functions


In [ ]:
# ============================
# 3.1 Helper Functions
# ============================

def _frame_num_for_path(frame_num: Union[str, int]) -> str:
    """Return zero-padded frame identifier for filesystem paths."""

    return str(frame_num).zfill(5)


def build_frame_tag(frame_num: Union[str, int]) -> str:
    """Return canonical frame identifier with 'F' prefix."""

    return f"F{_frame_num_for_path(frame_num)}"


def build_mintpy_output_dir(
    parent_dir: str,
    version: str,
    frame_num: Union[str, int],
) -> str:
    """Return MintPy output directory for the requested frame."""

    return os.path.join(
        parent_dir,
        version,
        build_frame_tag(frame_num),
        "mintpy_output",
    )


def _ensure_figures_dir(parent_dir: str, figures_dir: Optional[str] = None) -> str:
    """Return the figures directory path, creating it if needed."""

    target = figures_dir or "figures"
    if os.path.isabs(target):
        resolved = target
    else:
        resolved = os.path.join(parent_dir, target)
    os.makedirs(resolved, exist_ok=True)
    return resolved


def _display_figures_side_by_side(*figures, width: str = "48%") -> None:
    """Display matplotlib figures next to each other in the notebook."""

    if not figures:
        return

    parts = ["<div style='display:flex; gap:16px; flex-wrap:wrap;'>"]
    for fig in figures:
        buffer = BytesIO()
        fig.savefig(buffer, format="png", bbox_inches="tight")
        encoded = base64.b64encode(buffer.getvalue()).decode("ascii")
        parts.append(
            f"<img src='data:image/png;base64,{encoded}' "
            f"style='width:{width}; max-width:100%; height:auto;'/>"
        )
    parts.append("</div>")
    display(HTML("".join(parts)))


def _capture_view_figure(scp_args: str) -> Optional[plt.Figure]:
    """Run MintPy view for the supplied args and return the figure."""

    original_show = plt.show
    try:
        plt.show = lambda *_, **__: None
        existing = set(plt.get_fignums())
        view.main(scp_args.split())
        new_nums = [
            num for num in plt.get_fignums() if num not in existing
        ]
        target = (
            new_nums[-1]
            if new_nums
            else (plt.get_fignums()[-1] if plt.get_fignums() else None)
        )
        if target is None:
            return None
        return plt.figure(target)
    finally:
        plt.show = original_show


def build_file_path(
    parent_dir: str,
    version: str,
    frame_num: Union[str, int],
) -> str:
    """Return the path to the MintPy timeseries density file."""

    output_dir = build_mintpy_output_dir(parent_dir, version, frame_num)
    return os.path.join(output_dir, "timeseries_density.h5")


def load_density(
    file_path: str,
    threshold_density: Optional[float] = None,
) -> Tuple[np.ndarray, int]:
    """Load and filter MintPy timeseries density values."""

    with h5py.File(file_path, "r") as f:
        data = f["timeseriesdensity"][:]

    data = data.ravel()
    data = data[np.isfinite(data) & (data != 0)]

    if threshold_density is not None:
        data = data[data >= threshold_density]

    count_valid = np.count_nonzero(~np.isnan(data))
    return data, int(count_valid)


def make_output_name(
    parent_dir: str,
    prefix: str,
    frame_num: Union[str, int],
    version_num: str,
    version_to_comp: Optional[str],
    threshold_density: Optional[float],
    figures_dir: Optional[str] = None,
) -> str:
    """Return a descriptive PNG name in the shared figures directory."""

    frame_token = _frame_num_for_path(frame_num)
    parts = [f"{prefix}_{frame_token}_{version_num}"]
    if version_to_comp:
        parts.append(f"{version_to_comp}")
    if threshold_density is not None:
        parts.append(f"_thresh{threshold_density:g}")
    fname = "".join(parts) + ".png"
    resolved_figures_dir = _ensure_figures_dir(parent_dir, figures_dir)
    return os.path.join(resolved_figures_dir, fname)


def build_recommended_mask_path(
    parent_dir: str,
    version: str,
    frame_num: Union[str, int],
    threshold_density: float,
) -> str:
    """Return path to the recommended mask file for the given threshold."""

    frame_token = _frame_num_for_path(frame_num)
    thresh_pct = f"{threshold_density:.0%}".rstrip("%")
    return os.path.join(
        parent_dir,
        version,
        f"F{frame_token}",
        "mintpy_output",
        f"recommended_mask_{thresh_pct}thresh.h5",
    )


def capture_view(
    file_path: Optional[str],
    suffix: str,
) -> None:
    """Run MintPy view for a file and display it inline."""

    if not file_path:
        print("No file path supplied for capture_view.")
        return

    fig = _capture_view_figure(f"{file_path} {suffix}")
    if fig is None:
        print(f"No MintPy figure was produced for {file_path}")
        return

    display(fig)
    plt.close(fig)


def build_epoch_recommended_mask_path(
    parent_dir: str,
    version: str,
    frame_num: Union[str, int],
) -> str:
    """Return the per-epoch recommended mask path within MintPy output."""

    output_dir = os.path.dirname(
        build_file_path(
            parent_dir=parent_dir,
            version=version,
            frame_num=frame_num,
        )
    )
    return os.path.join(output_dir, "recommended_mask.h5")


def build_timeseries_path(
    parent_dir: str,
    version: str,
    frame_num: Union[str, int],
) -> str:
    """Return the MintPy timeseries path for the requested frame."""

    return build_mintpy_output_path(
        parent_dir=parent_dir,
        version=version,
        frame_num=frame_num,
        filename="timeseries.h5",
    )


def build_mintpy_output_path(
    parent_dir: str,
    version: str,
    frame_num: Union[str, int],
    filename: str,
) -> str:
    """Return a MintPy output path for the specified filename."""

    output_dir = build_mintpy_output_dir(parent_dir, version, frame_num)
    return os.path.join(output_dir, filename)


def build_epoch_layer_paths(
    filename: str,
    parent_dir: str,
    frame_num: Union[str, int],
    primary_version: str,
    comparison_version: Optional[str] = None,
) -> Tuple[str, Optional[str]]:
    """Return the primary and comparison MintPy paths for a filename."""

    primary_path = build_mintpy_output_path(
        parent_dir=parent_dir,
        version=primary_version,
        frame_num=frame_num,
        filename=filename,
    )
    compare_path = None
    if comparison_version:
        compare_path = build_mintpy_output_path(
            parent_dir=parent_dir,
            version=comparison_version,
            frame_num=frame_num,
            filename=filename,
        )
    return primary_path, compare_path


def ensure_file_exists(file_path: str, descriptor: str) -> None:
    """Raise a descriptive error when an expected file path is missing."""

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Missing {descriptor}: {file_path}")


def compute_displacement_range(
    timeseries_path: str,
    dataset: str = "timeseries",
    scale: float = 100.0,
) -> Tuple[float, float]:
    """Return min/max values scaled to the requested units."""

    subdataset = f'HDF5:"{timeseries_path}"://{dataset}'
    ds = gdal.Open(subdataset, gdal.GA_ReadOnly)
    if ds is None:
        raise RuntimeError(f"Could not open dataset: {subdataset}")

    arr = ds.ReadAsArray().astype(float)
    if arr.ndim == 2:
        arr = arr[np.newaxis, :, :]

    vmin = float(np.nanmin(arr)) * scale
    vmax = float(np.nanmax(arr)) * scale
    ds = None
    return vmin, vmax





_COMBINED_RANGE_SUMMARY: List[Dict[str, Any]] = []

_RANGE_SUMMARY_METADATA: Dict[str, Dict[str, str]] = {
    "Recommended Mask": {
        "label": "4.1 Recommended Mask",
        "anchor": "#41-recommended-mask",
        "units": "mask",
    },
    "Displacement": {
        "label": "4.2 Displacement",
        "anchor": "#42-displacement",
        "units": "cm",
    },
    "Short Wavelength Displacement": {
        "label": "4.3 Short Wavelength Displacement",
        "anchor": "#43-short-wavelength-displacement",
        "units": "cm",
    },
    "Ionospheric Delay": {
        "label": "4.4 Ionospheric Delay",
        "anchor": "#44-ionospheric-delay",
        "units": "cm",
    },
    "Solid Earth Tide": {
        "label": "4.5 Solid Earth Tide",
        "anchor": "#45-solid-earth-tide",
        "units": "cm",
    },
    "Perpendicular Baseline": {
        "label": "4.6 Perpendicular Baseline",
        "anchor": "#46-perpendicular-baseline",
        "units": "cm",
    },
    "Connected Component Labels": {
        "label": "4.7 Connected Component Labels",
        "anchor": "#47-connected-component-labels",
        "units": "unitless",
    },
    "Shape Counts": {
        "label": "4.8 Shape Counts",
        "anchor": "#48-shape-counts",
        "units": "unitless",
    },
    "Temporal Coherence": {
        "label": "4.9 Temporal Coherence",
        "anchor": "#49-temporal-coherence",
        "units": "unitless",
    },
    "Estimated Phase Quality": {
        "label": "4.10 Estimated Phase Quality",
        "anchor": "#410-estimated-phase-quality",
        "units": "unitless",
    },
    "Phase Similarity": {
        "label": "4.11 Phase Similarity",
        "anchor": "#411-phase-similarity",
        "units": "unitless",
    },
    "Water Mask": {
        "label": "4.12 Water Mask",
        "anchor": "#412-water-mask",
        "units": "mask",
    },
    "Timeseries Inversion Residuals": {
        "label": "4.13 Timeseries Inversion Residuals",
        "anchor": "#413-timeseries-inversion-residuals",
        "units": "cm",
    },
    "DEM Error": {
        "label": "4.14 DEM Error",
        "anchor": "#414-dem-error",
        "units": "cm",
    },
    "Raw Velocity": {
        "label": "5.1 Raw Velocity",
        "anchor": "#51-raw-velocity",
        "units": "cm/year",
    },
    "DEM Error Corrected Velocity": {
        "label": "5.2 DEM Error Corrected Velocity",
        "anchor": "#52-dem-error-corrected-velocity",
        "units": "cm/year",
    },
    "Short Wavelength Velocity": {
        "label": "5.3 Short Wavelength Velocity",
        "anchor": "#53-short-wavelength-velocity",
        "units": "cm/year",
    },
}


def _infer_units_from_description(scale: float, description: str) -> str:
    """Return default units based on scale and descriptor text."""

    lowered = description.lower()
    if scale == 100.0:
        if 'velocity' in lowered:
            return 'cm/year'
        return 'cm'
    if scale == 1.0:
        if 'mask' in lowered:
            return 'mask'
        return 'unitless'
    return 'unitless'


def report_combined_range(
    description: str,
    primary_path: Optional[Union[str, Path]],
    comparison_path: Optional[Union[str, Path]],
    dataset: Union[str, Sequence[str]] = 'timeseries',
    scale: float = 100.0,
    primary_range: Optional[Tuple[float, float]] = None,
    comparison_range: Optional[Tuple[float, float]] = None,
    units: Optional[str] = None,
    summary_label: Optional[str] = None,
    summary_anchor: Optional[str] = None,
) -> None:
    """Print and record the collective range across the supplied files."""

    dataset_options: Tuple[str, ...]
    if isinstance(dataset, str):
        dataset_options = (dataset,)
    else:
        dataset_options = tuple(dataset)

    metadata_entry = _RANGE_SUMMARY_METADATA.get(description, {})
    resolved_label = (
        summary_label or metadata_entry.get('label') or description
    )
    resolved_anchor = summary_anchor or metadata_entry.get('anchor')
    resolved_units = (
        units
        or metadata_entry.get('units')
        or _infer_units_from_description(scale, description)
    )

    entries: List[Tuple[str, str, float, float]] = []
    sources = [
        ('primary', primary_path, primary_range),
        ('comparison', comparison_path, comparison_range),
    ]
    for label, path, cached in sources:
        if not path:
            continue
        if not os.path.exists(path):
            print(
                f"Missing {description} {label} file for combined range: "
                f"{path}"
            )
            continue
        current_range = cached
        if current_range is None:
            last_error: Optional[str] = None
            for dataset_name in dataset_options:
                try:
                    current_range = compute_displacement_range(
                        path,
                        dataset=dataset_name,
                        scale=scale,
                    )
                    break
                except Exception as exc:  # pragma: no cover - logging path
                    last_error = str(exc)
                    continue
            if current_range is None:
                print(
                    f"Unable to compute range for {description} {label} file "
                    f"{path}: {last_error or 'unknown error'}"
                )
                continue
        entries.append((label, path, current_range[0], current_range[1]))

    if not entries:
        print(
            f"No available files to report combined range for {description}."
        )
        return

    combined_min = min(item[2] for item in entries)
    combined_max = max(item[3] for item in entries)
    print(
        f"{description} combined range in ({resolved_units}) "
        f"across {len(entries)} file(s): "
        f"min={combined_min:.3f}, max={combined_max:.3f}"
    )

    _COMBINED_RANGE_SUMMARY.append(
        {
            'label': resolved_label,
            'anchor': resolved_anchor,
            'units': resolved_units,
            'min': combined_min,
            'max': combined_max,
        }
    )


def read_timeseries_dates(timeseries_path: str) -> List[str]:
    """Return the date strings stored in the MintPy timeseries file."""

    with h5py.File(timeseries_path, "r") as f:
        raw_dates = f["date"][:]
    return [d.decode("utf-8") for d in raw_dates]


def compute_common_reference_date(
    primary_path: str,
    compare_path: Optional[str],
) -> Optional[str]:
    """Return the earliest common date shared by both timeseries."""

    primary_dates = read_timeseries_dates(primary_path)
    if not primary_dates:
        return None

    if compare_path and os.path.exists(compare_path):
        compare_dates = read_timeseries_dates(compare_path)
        common = sorted(set(primary_dates) & set(compare_dates))
        if common:
            return common[0]

    return min(primary_dates)



_COMMON_VIEW_DATE_CACHE: Dict[Tuple[str, str], List[str]] = {}


def get_common_date_list(
    primary_path: Optional[Union[str, Path]],
    comparison_path: Optional[Union[str, Path]],
) -> List[str]:
    """Return sorted common acquisition dates shared by two files."""

    if not primary_path or not comparison_path:
        return []

    resolved_primary = os.path.abspath(str(primary_path))
    resolved_comparison = os.path.abspath(str(comparison_path))
    cache_key = tuple(sorted([resolved_primary, resolved_comparison]))
    cached = _COMMON_VIEW_DATE_CACHE.get(cache_key)
    if cached is not None:
        return cached

    if not (
        os.path.exists(resolved_primary)
        and os.path.exists(resolved_comparison)
    ):
        _COMMON_VIEW_DATE_CACHE[cache_key] = []
        return []

    try:
        primary_dates = read_timeseries_dates(resolved_primary)
        comparison_dates = read_timeseries_dates(resolved_comparison)
    except (OSError, KeyError) as exc:
        print(
            "Unable to derive common dates for "
            f"{resolved_primary} vs {resolved_comparison}: {exc}"
        )
        _COMMON_VIEW_DATE_CACHE[cache_key] = []
        return []

    common_dates = sorted(set(primary_dates) & set(comparison_dates))
    if not common_dates:
        print(
            "No overlapping acquisition dates for "
            f"{resolved_primary} vs {resolved_comparison}."
        )
    _COMMON_VIEW_DATE_CACHE[cache_key] = common_dates
    return common_dates


def build_common_date_argument(
    primary_path: Optional[Union[str, Path]],
    comparison_path: Optional[Union[str, Path]],
) -> str:
    """Return a --date-list argument for MintPy view when possible."""

    common_dates = get_common_date_list(primary_path, comparison_path)
    if not common_dates:
        return ""
    joined = " ".join(common_dates)
    return f"{joined}"




def format_common_dates_for_view(arg: Optional[str]) -> str:
    """Return the formatted --date-list clause for capture_view."""

    return f"{arg} " if arg else ""


def _dates_to_fractional_years(
    date_strings: Sequence[str],
) -> np.ndarray:
    """Return decimal-year offsets relative to the first timestamp."""

    parsed_dates = [
        dt.strptime(str(item), "%Y%m%d") for item in date_strings
    ]
    start = parsed_dates[0]
    seconds_per_year = 365.25 * 24 * 3600.0
    return np.array(
        [
            (current - start).total_seconds() / seconds_per_year
            for current in parsed_dates
        ],
        dtype=np.float64,
    )


def _fit_velocity_block(
    data_block: np.ndarray,
    time_axis: np.ndarray,
) -> np.ndarray:
    """Fit a linear velocity for every pixel in the data_block."""

    num_times, rows, cols = data_block.shape
    reshaped = data_block.reshape(num_times, rows * cols).astype(np.float64)
    mask = np.isfinite(reshaped)
    if not np.any(mask):
        return np.full((rows, cols), np.nan, dtype=np.float32)

    weights = mask.astype(np.float64)
    weighted_values = np.where(mask, reshaped, 0.0)
    t_column = time_axis[:, None]

    sum_w = weights.sum(axis=0)
    valid = sum_w >= 2
    sum_x = np.sum(weights * t_column, axis=0)
    sum_y = np.sum(weights * weighted_values, axis=0)
    sum_xx = np.sum(weights * (t_column ** 2), axis=0)
    sum_xy = np.sum(weights * t_column * weighted_values, axis=0)
    denom = sum_w * sum_xx - sum_x ** 2

    slopes = np.full(reshaped.shape[1], np.nan, dtype=np.float64)
    good = valid & (np.abs(denom) > 0)
    slopes[good] = (
        (sum_w[good] * sum_xy[good] - sum_x[good] * sum_y[good])
        / denom[good]
    )
    return slopes.reshape(rows, cols).astype(np.float32)


def compute_velocity_from_timeseries(
    timeseries_path: Union[str, Path],
    selected_dates: Optional[Sequence[str]],
    output_filename: str,
    description: str,
) -> str:
    """Compute a velocity file limited to the provided acquisition dates."""

    ts_path = Path(timeseries_path)
    if not ts_path.exists():
        raise FileNotFoundError(f"Missing timeseries file: {ts_path}")

    metadata = readfile.read_attribute(str(ts_path))
    output_path = ts_path.with_name(output_filename)
    if output_path.exists():
        print(f"Reusing cached velocity file at {output_path}")
        return str(output_path)

    with h5py.File(ts_path, "r") as ts_file:
        all_dates = [d.decode("utf-8") for d in ts_file["date"][:]]
        if not all_dates:
            raise ValueError(
                f"{ts_path} does not contain any acquisition dates."
            )

        lookup = {date: idx for idx, date in enumerate(all_dates)}
        if selected_dates:
            normalized: List[str] = []
            seen = set()
            for date in selected_dates:
                date_str = str(date)
                if date_str in seen:
                    continue
                seen.add(date_str)
                normalized.append(date_str)
            missing = [date for date in normalized if date not in lookup]
            if missing:
                raise ValueError(
                    "Requested dates not found in "
                    f"{ts_path}: {', '.join(missing)}"
                )
            selected = sorted(normalized, key=lambda item: lookup[item])
        else:
            selected = list(all_dates)

        if len(selected) < 2:
            raise ValueError(
                "At least two dates are required for velocity fitting."
            )

        selected_key = ",".join(selected)
        if output_path.exists():
            try:
                existing_attr = readfile.read_attribute(str(output_path))
                if existing_attr.get("COMMON_DATES") == selected_key:
                    print(
                        "Reusing cached velocity file at "
                        f"{output_path}"
                    )
                    return str(output_path)
            except Exception:
                pass

        date_indices = [lookup[date] for date in selected]
        time_axis = _dates_to_fractional_years(selected)

        dataset = ts_file["timeseries"]
        length = dataset.shape[1]
        width = dataset.shape[2]
        velocities = np.full((length, width), np.nan, dtype=np.float32)
        chunk_rows = 256

        for row_start in range(0, length, chunk_rows):
            row_stop = min(length, row_start + chunk_rows)
            block = dataset[date_indices, row_start:row_stop, :]
            velocities[row_start:row_stop, :] = _fit_velocity_block(
                block,
                time_axis,
            )

    ref_y = metadata.get("REF_Y")
    ref_x = metadata.get("REF_X")
    try:
        ref_y_idx = int(float(ref_y))
        ref_x_idx = int(float(ref_x))
    except (TypeError, ValueError):
        ref_y_idx = ref_x_idx = None

    if (
        ref_y_idx is not None
        and ref_x_idx is not None
        and 0 <= ref_y_idx < velocities.shape[0]
        and 0 <= ref_x_idx < velocities.shape[1]
    ):
        ref_value = velocities[ref_y_idx, ref_x_idx]
        if np.isfinite(ref_value):
            velocities -= ref_value

    start_date = selected[0]
    end_date = selected[-1]
    metadata = dict(metadata)
    metadata.update(
        {
            "DATA_TYPE": "float32",
            "FILE_TYPE": "velocity",
            "FILE_PATH": str(ts_path),
            "START_DATE": start_date,
            "END_DATE": end_date,
            "REF_DATE": start_date,
            "DATE12": f"{start_date}_{end_date}",
            "UNIT": "m/year",
            "COMMON_DATES": selected_key,
            "DESCRIPTION": description,
        }
    )

    ds_name = {"velocity": [np.float32, velocities.shape, None]}
    ensure_directory(output_path.parent)
    print(
        f"Computing velocity file {output_path} using "
        f"{len(selected)} dates."
    )
    writefile.layout_hdf5(str(output_path), ds_name, metadata=metadata)
    with h5py.File(output_path, "a") as out_file:
        out_file["velocity"][:] = velocities

    return str(output_path)


def ensure_directory(path: Union[str, Path]) -> str:
    """Create a directory path if needed and return its string form."""

    resolved = Path(path)
    resolved.mkdir(parents=True, exist_ok=True)
    return str(resolved)


def read_lat_lon_wkt(
wkt_path: Union[str, Path],
) -> List[Tuple[float, float]]:
    """Return (lat, lon) tuples stored in POINT or MULTIPOINT WKT."""

    path = Path(wkt_path)
    if not path.exists():
        raise FileNotFoundError(f"Missing WKT file: {path}")

    text = path.read_text().strip()
    if not text:
        return []

    upper_text = text.upper()
    if upper_text.startswith("MULTIPOINT"):
        inner = text[text.find("(") + 1 : text.rfind(")")]
        chunks = [chunk.strip() for chunk in inner.split("),")]
        points: List[Tuple[float, float]] = []
        for chunk in chunks:
            cleaned = chunk.replace("(", "").replace(")", "").strip()
            if not cleaned:
                continue
            lon_str, lat_str = cleaned.split()
            points.append((float(lat_str), float(lon_str)))
        return points

    if upper_text.startswith("POINT"):
        inner = text[text.find("(") + 1 : text.rfind(")")]
        lon_str, lat_str = inner.strip().split()
        return [(float(lat_str), float(lon_str))]

    raise ValueError(
        "Unsupported WKT geometry; expected POINT or MULTIPOINT entries."
    )


def write_lat_lon_wkt(
    points: Sequence[Tuple[float, float]],
    output_path: Union[str, Path],
    merge_existing: bool = False,
) -> str:
    """Persist latitude/longitude points as a MULTIPOINT WKT file."""

    path = Path(output_path)
    ensure_directory(path.parent)
    all_points: List[Tuple[float, float]] = []
    if merge_existing and path.exists():
        all_points.extend(read_lat_lon_wkt(path))
    all_points.extend(points)

    unique_points: List[Tuple[float, float]] = []
    seen = set()
    for lat, lon in all_points:
        key = (round(lat, 6), round(lon, 6))
        if key in seen:
            continue
        seen.add(key)
        unique_points.append((lat, lon))

    if not unique_points:
        wkt_text = "MULTIPOINT EMPTY"
    else:
        coords = ", ".join(
            f"({lon:.6f} {lat:.6f})" for lat, lon in unique_points
        )
        wkt_text = f"MULTIPOINT ({coords})"
    path.write_text(wkt_text)
    return str(path)


def resolve_lat_lon_input(
    lat_lon_param: Union[str, Path, Sequence[Sequence[float]], int, None]
) -> Tuple[List[Tuple[float, float]], int, Optional[str]]:
    """Normalize lat/lon input into explicit points or a random count."""

    if lat_lon_param is None:
        return [], 0, None

    if isinstance(lat_lon_param, (str, Path)):
        points = read_lat_lon_wkt(lat_lon_param)
        return points, 0, str(lat_lon_param)

    if isinstance(lat_lon_param, int):
        if lat_lon_param <= 0:
            raise ValueError(
            "Integer lat/lon specification must be positive."
        )
        return [], lat_lon_param, None

    if isinstance(lat_lon_param, Sequence):
        normalized: List[Tuple[float, float]] = []
        for item in lat_lon_param:
            if not isinstance(item, Sequence) or len(item) != 2:
                raise ValueError(
                    "Lat/lon sequences must be numeric (lat, lon) pairs."
                )
            lat, lon = float(item[0]), float(item[1])
            normalized.append((lat, lon))
        return normalized, 0, None

    raise TypeError(
        "list_lat_lon must be a path, pair sequence, "
        "integer count, or None."
    )


def latlon_to_pixel(
lat: float,
lon: float,
metadata: dict,
) -> Tuple[int, int]:
    """Return (col, row) pixel indices for a latitude/longitude pair."""

    transformer = Transformer.from_crs(
        "EPSG:4326",
        f"EPSG:{metadata['EPSG']}",
        always_xy=True,
    )
    x_utm, y_utm = transformer.transform(lon, lat)
    col_idx = int(
        (x_utm - float(metadata['X_FIRST'])) / float(metadata['X_STEP'])
    )
    row_idx = int(
        (y_utm - float(metadata['Y_FIRST'])) / float(metadata['Y_STEP'])
    )
    return col_idx, row_idx




def plot_height_map_with_points(
    geometry_path: Union[str, Path],
    metadata: dict,
    points: Sequence[Tuple[float, float]],
    title: str,
    output_path: Optional[Union[str, Path]] = None,
    highlight_info: Optional[Sequence[Tuple[int, float, float]]] = None,
    highlight_color: str = 'red',
) -> None:
    """Plot a height map with numbered points and optional highlights."""

    geometry_path = str(geometry_path)
    with h5py.File(geometry_path, 'r') as geom_file:
        if 'height' not in geom_file:
            raise KeyError("Dataset '/height' not found in geometry file.")
        height_field = geom_file['height'][:]

    cmap = plt.get_cmap('terrain')
    vmin = float(np.nanpercentile(height_field, 2))
    vmax = float(np.nanpercentile(height_field, 98))

    fig, ax = plt.subplots(figsize=[12, 12])
    im = ax.imshow(
        height_field,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        interpolation='nearest',
    )
    cbar = fig.colorbar(
        im,
        ax=ax,
        orientation='vertical',
        pad=0.02,
        shrink=0.5,
    )
    cbar.set_label('Height [m]')

    for idx, (lat, lon) in enumerate(points, start=1):
        col_idx, row_idx = latlon_to_pixel(lat, lon, metadata)
        ax.plot(col_idx, row_idx, 'ko', markersize=5)
        ax.text(
            col_idx + 30,
            row_idx + 30,
            str(idx),
            fontsize=24,
            fontweight='bold',
            ha='left',
            va='bottom',
            bbox=dict(
                facecolor='white',
                alpha=0.5,
                edgecolor='none',
                boxstyle='round',
                pad=0.3,
            ),
        )

    if highlight_info:
        for point_idx, lat, lon in highlight_info:
            col_idx, row_idx = latlon_to_pixel(lat, lon, metadata)
            ax.plot(
                col_idx,
                row_idx,
                'o',
                markersize=10,
                markerfacecolor='none',
                markeredgecolor=highlight_color,
                markeredgewidth=2,
            )
            ax.text(
                col_idx + 30,
                row_idx - 20,
                str(point_idx),
                fontsize=30,
                fontweight='bold',
                color=highlight_color,
                ha='left',
                va='bottom',
                bbox=dict(
                    facecolor='white',
                    alpha=0.7,
                    edgecolor='none',
                    boxstyle='round',
                    pad=0.3,
                ),
            )

    ax.set_title(title)
    ax.axis('off')

    if output_path:
        output_path = Path(output_path)
        ensure_directory(output_path.parent)
        fig.savefig(output_path, bbox_inches='tight', dpi=300)
    display(fig)
    plt.close(fig)

_COMMON_DATES_LOGGED = False
_COMMON_DATES_CACHE: Dict[Tuple[str, str], Dict[str, Any]] = {}


def reset_common_dates_cache() -> None:
    """Clear cached common date metadata between comparison runs."""

    global _COMMON_DATES_LOGGED, _COMMON_DATES_CACHE
    _COMMON_DATES_LOGGED = False
    _COMMON_DATES_CACHE = {}


def _parse_ref_lalo(value: Optional[str]) -> Optional[Tuple[float, float]]:
    """Parse ref_lalo string into (lat, lon)."""

    if value is None:
        return None
    if not isinstance(value, str):
        raise TypeError("ref_lalo must be a string like 'lat lon' or None.")
    parts = value.replace(',', ' ').split()
    if len(parts) != 2:
        raise ValueError("ref_lalo must contain two values: 'lat lon'.")
    return float(parts[0]), float(parts[1])


def _point_has_valid_timeseries(
    timeseries_path: Union[str, Path],
    row_idx: int,
    col_idx: int,
) -> bool:
    """Return True if the pixel has any finite nonzero observations."""

    ts_path = Path(timeseries_path)
    if not ts_path.exists():
        raise FileNotFoundError(f"Missing timeseries file: {ts_path}")

    atr = readfile.read_attribute(str(ts_path))
    width = int(atr.get('WIDTH') or atr.get('width'))
    length = int(atr.get('LENGTH') or atr.get('length'))
    if not (0 <= col_idx < width and 0 <= row_idx < length):
        raise ValueError(
            f"Reference point out of bounds: row={row_idx}, col={col_idx}"
        )

    data, _ = readfile.read(
        str(ts_path),
        datasetName='timeseries',
        box=(col_idx, row_idx, col_idx + 1, row_idx + 1),
    )
    data = np.asarray(data)
    if data.ndim < 1:
        raise ValueError(
            f"Unexpected timeseries shape for reference check: {data.shape}"
        )
    series = data.reshape(data.shape[0], -1)
    return bool(np.any(np.isfinite(series) & (series != 0)))


def _reference_point_attribute(
    atr: Dict[str, Any],
    y: int,
    x: int,
) -> Dict[str, str]:
    """Return updated reference point attributes."""

    atr_new = {
        'REF_Y': str(y),
        'REF_X': str(x),
    }
    coord = ut.coordinate(atr)
    if 'X_FIRST' in atr.keys():
        ref_lat, ref_lon = coord.yx2lalo(y, x)
        atr_new['REF_LAT'] = str(ref_lat)
        atr_new['REF_LON'] = str(ref_lon)
    return atr_new


def _update_reference_in_file(
    file_path: Union[str, Path],
    ref_y: int,
    ref_x: int,
) -> None:
    """Apply spatial referencing relative to (ref_y, ref_x) in-place."""

    path = Path(file_path)
    if not path.exists():
        print(f"Missing file for reference update: {path}")
        return

    atr = readfile.read_attribute(str(path))
    ftype = atr.get('FILE_TYPE')
    if not ftype:
        raise ValueError(f"FILE_TYPE missing in attributes for {path}")

    atr_new = _reference_point_attribute(atr, y=ref_y, x=ref_x)

    with h5py.File(path, 'r+') as h5:
        if ftype not in h5:
            raise ValueError(
                f"Dataset '{ftype}' not found in {path}"
            )
        ds = h5[ftype]
        if ds.ndim == 3:
            for i in range(ds.shape[0]):
                ref_val = ds[i, ref_y, ref_x]
                if not np.isfinite(ref_val) or ref_val == 0:
                    continue
                data_2d = ds[i, :, :]
                mask = np.isfinite(data_2d) & (data_2d != 0)
                data_2d[mask] = data_2d[mask] - ref_val
                ds[i, :, :] = data_2d
        else:
            ref_val = ds[ref_y, ref_x]
            if np.isfinite(ref_val) and ref_val != 0:
                data_2d = ds[:, :]
                mask = np.isfinite(data_2d) & (data_2d != 0)
                data_2d[mask] = data_2d[mask] - ref_val
                ds[:, :] = data_2d
            else:
                print(f"Skipping reference subtraction for {path}; ref pixel is 0 or NaN.")

        h5.attrs.update(atr_new)


def apply_reference_if_requested(
    ref_lalo: Optional[str],
    timeseries_path: Union[str, Path],
    primary_dir: str,
    comparison_dir: Optional[str],
    metadata: Dict[str, Any],
) -> None:
    """Update reference point across MintPy outputs if ref_lalo is set."""

    parsed = _parse_ref_lalo(ref_lalo)
    if parsed is None:
        return

    ref_lat, ref_lon = parsed
    atr = readfile.read_attribute(str(timeseries_path))

    coord = ut.coordinate(atr)
    test_ref_y, test_ref_x = coord.geo2radar(
        np.array(float(ref_lat)),
        np.array(float(ref_lon)),
    )[0:2]
    test_ref_y = int(round(float(test_ref_y)))
    test_ref_x = int(round(float(test_ref_x)))

    width = int(atr.get('WIDTH') or atr.get('width'))
    length = int(atr.get('LENGTH') or atr.get('length'))
    if not (0 <= test_ref_x < width and 0 <= test_ref_y < length):
        raise ValueError(
            f"Reference point out of bounds: row={test_ref_y}, col={test_ref_x}"
        )

    ref_y = metadata.get('REF_Y')
    ref_x = metadata.get('REF_X')
    if ref_y is not None:
        ref_y = int(ref_y)
    if ref_x is not None:
        ref_x = int(ref_x)

    if ref_y == test_ref_y and ref_x == test_ref_x:
        return

    if not _point_has_valid_timeseries(
        timeseries_path,
        row_idx=test_ref_y,
        col_idx=test_ref_x,
    ):
        raise ValueError(
            "Specified reference point does not correspond to any "
            "nonzero finite data in the timeseries stack."
        )

    files_to_update = [
        'demErr.h5',
        'estimated_phase_quality.h5',
        'ionospheric_delay.h5',
        'perpendicular_baseline.h5',
        'phase_similarity.h5',
        'short_wavelength_displacement.h5',
        'solid_earth_tide.h5',
        'timeseries_demErr.h5',
        'timeseries.h5',
        'timeseries_inversion_residuals.h5',
        'timeseriesResidual.h5',
        'velocity_demErr.h5',
        'velocity.h5',
        'velocity_shortwvl.h5',
    ]

    output_dirs = [primary_dir]
    if comparison_dir:
        output_dirs.append(comparison_dir)

    for out_dir in output_dirs:
        for fname in files_to_update:
            _update_reference_in_file(Path(out_dir) / fname, test_ref_y, test_ref_x)


def _load_optional_mask_indices(primary_dir: str) -> Optional[np.ndarray]:
    """Return array of [row, col] indices for valid mask pixels."""

    mask_files = sorted(Path(primary_dir).glob('*_mask.tif'))
    if not mask_files:
        return None

    mask_path = mask_files[0]
    ds = gdal.Open(str(mask_path), gdal.GA_ReadOnly)
    if ds is None:
        print(f"Unable to open mask file: {mask_path}")
        return None

    data = ds.ReadAsArray()
    ds = None
    if data is None:
        print(f"Mask file {mask_path} contains no data.")
        return None

    mask = np.isfinite(data) & (data != 0)
    indices = np.column_stack(np.where(mask))
    if indices.size == 0:
        print(f"Mask file {mask_path} has no valid nonzero pixels.")
        return None
    return indices


def extract_timeseries_data(
    primary_dir: str,
    comparison_dir: str,
    ts_filename: str = "timeseries.h5",
    pixel_location: Optional[Tuple[int, int]] = None,
    radius: int = 3,
    rng: Optional[np.random.Generator] = None,
    subtract_initial_epoch: bool = True,
    allow_random_fallback: bool = True,
) -> Tuple[
    List[dt],
    np.ndarray,
    np.ndarray,
    Tuple[float, float],
    Tuple[int, int],
    Optional[str],
]:
    """Return aligned displacement stacks for the specified pixel."""

    global _COMMON_DATES_LOGGED, _COMMON_DATES_CACHE

    if comparison_dir is None:
        raise ValueError(
            "comparison_dir is required for time series comparison.",
        )

    if pixel_location is None and not allow_random_fallback:
        raise ValueError(
            "pixel_location is required when allow_random_fallback is False."
        )

    primary_path = os.path.abspath(os.path.join(primary_dir, ts_filename))
    comparison_path = os.path.abspath(
        os.path.join(comparison_dir, ts_filename)
    )

    cache_key = (primary_path, comparison_path)
    cache_entry = _COMMON_DATES_CACHE.get(cache_key)

    if cache_entry is None:
        primary_dates = timeseries(primary_path).get_date_list()
        comparison_dates = timeseries(comparison_path).get_date_list()
        common_dates = sorted(set(primary_dates) & set(comparison_dates))
        if not common_dates:
            raise ValueError(
                "No overlapping acquisition dates between the time series."
            )

        primary_indices = [
            primary_dates.index(date) for date in common_dates
        ]
        comparison_indices = [
            comparison_dates.index(date) for date in common_dates
        ]
        cache_entry = {
            "common_dates": common_dates,
            "primary_indices": primary_indices,
            "comparison_indices": comparison_indices,
            "dates_dt": [
                dt.strptime(date, "%Y%m%d")
                for date in common_dates
            ],
        }
        _COMMON_DATES_CACHE[cache_key] = cache_entry
        if not _COMMON_DATES_LOGGED:
            print(f"Common dates: {common_dates}")
            _COMMON_DATES_LOGGED = True

    common_dates = cache_entry["common_dates"]
    primary_indices = cache_entry["primary_indices"]
    comparison_indices = cache_entry["comparison_indices"]
    dates = cache_entry["dates_dt"]

    primary_attr = readfile.read_attribute(primary_path)
    width = int(primary_attr.get('WIDTH') or primary_attr.get('width'))
    length = int(primary_attr.get('LENGTH') or primary_attr.get('length'))

    transformer = Transformer.from_crs(
        f"EPSG:{primary_attr['EPSG']}",
        "EPSG:4326",
        always_xy=True,
    )

    rng = rng or np.random.default_rng()

    def read_window(col_idx: int, row_idx: int):
        if not (0 <= col_idx < width and 0 <= row_idx < length):
            return None, (
                f"Pixel Row:{row_idx}, Col:{col_idx} is out of bounds."
            )

        x0 = max(0, col_idx - radius)
        x1 = min(width, col_idx + radius + 1)
        y0 = max(0, row_idx - radius)
        y1 = min(length, row_idx + radius + 1)
        box = (x0, y0, x1, y1)

        primary_window, _ = readfile.read(
            primary_path,
            datasetName='timeseries',
            box=box,
        )
        comparison_window, _ = readfile.read(
            comparison_path,
            datasetName='timeseries',
            box=box,
        )

        primary_window = primary_window[primary_indices, :, :]
        comparison_window = comparison_window[comparison_indices, :, :]

        if subtract_initial_epoch:
            primary_window = (
                primary_window - primary_window[0][np.newaxis, :, :]
            )
            comparison_window = (
                comparison_window - comparison_window[0][np.newaxis, :, :]
            )

        primary_valid = (
            np.isfinite(primary_window) & (primary_window != 0)
        )
        comparison_valid = (
            np.isfinite(comparison_window) & (comparison_window != 0)
        )
        joint_valid = primary_valid & comparison_valid

        center_row = row_idx - y0
        center_col = col_idx - x0
        center_joint = joint_valid[:, center_row, center_col]
        if not np.any(center_joint):
            return None, (
                f"Pixel Row:{row_idx}, Col:{col_idx} has no overlapping "
                "nonzero finite observations in the MintPy stacks."
            )

        primary_values = np.zeros(len(common_dates))
        comparison_values = np.zeros(len(common_dates))

        for t in range(len(common_dates)):
            time_valid = joint_valid[t]
            if not np.any(time_valid):
                primary_values[t] = np.nan
                comparison_values[t] = np.nan
                continue
            window_primary = primary_window[t]
            window_comparison = comparison_window[t]
            primary_values[t] = float(np.mean(window_primary[time_valid]))
            comparison_values[t] = float(
                np.mean(window_comparison[time_valid])
            )

        x_coord = (
            float(primary_attr['X_FIRST'])
            + col_idx * float(primary_attr['X_STEP'])
        )
        y_coord = (
            float(primary_attr['Y_FIRST'])
            + row_idx * float(primary_attr['Y_STEP'])
        )
        lon, lat = transformer.transform(x_coord, y_coord)
        return (
            primary_values,
            comparison_values,
            (lat, lon),
            (col_idx, row_idx),
        ), None

    mask_indices = _load_optional_mask_indices(primary_dir)
    def random_pixel() -> Tuple[int, int]:
        if mask_indices is not None:
            idx = int(rng.integers(mask_indices.shape[0]))
            row_idx, col_idx = mask_indices[idx]
            return int(col_idx), int(row_idx)
        return int(rng.integers(width)), int(rng.integers(length))

    selection_note: Optional[str] = None
    candidate = pixel_location
    attempts = 0

    try:
        primary_min, primary_max = compute_displacement_range(
            primary_path, dataset='timeseries', scale=1.0
        )
        comparison_min, comparison_max = compute_displacement_range(
            comparison_path, dataset='timeseries', scale=1.0
        )
        if (
            np.isclose(primary_min, 0.0)
            and np.isclose(primary_max, 0.0)
            and np.isclose(comparison_min, 0.0)
            and np.isclose(comparison_max, 0.0)
        ):
            warnings.warn(
                'Both primary and comparison timeseries ranges are 0; '
                'skipping random pixel selection.',
                UserWarning,
            )
            nan_series = np.full(len(dates), np.nan)
            return (
                dates,
                nan_series,
                nan_series.copy(),
                (np.nan, np.nan),
                (-1, -1),
                'no_valid_pixel',
            )
    except Exception as exc:
        warnings.warn(
            f'Range check failed ({exc}); skipping random pixel selection.',
            UserWarning,
        )
        nan_series = np.full(len(dates), np.nan)
        return (
            dates,
            nan_series,
            nan_series.copy(),
            (np.nan, np.nan),
            (-1, -1),
            'no_valid_pixel',
        )
    max_attempts = 1 if (candidate is not None and not allow_random_fallback) else 50

    while attempts < max_attempts:
        if candidate is None:
            col_idx, row_idx = random_pixel()
        else:
            col_idx, row_idx = int(candidate[0]), int(candidate[1])

        result, reason = read_window(col_idx, row_idx)
        if result is not None:
            (
                primary_values,
                comparison_values,
                lat_lon,
                pixel_indices,
            ) = result
            selection_note = None
            return (
                dates,
                primary_values,
                comparison_values,
                lat_lon,
                pixel_indices,
                selection_note,
            )

        if reason:
            print(reason)

        if not allow_random_fallback:
            raise ValueError(reason or "Pixel selection failed.")

        candidate = None
        attempts += 1

    warnings.warn(
        "Failed to find a valid pixel after multiple attempts.",
        UserWarning,
    )
    nan_series = np.full(len(dates), np.nan)
    return (
        dates,
        nan_series,
        nan_series.copy(),
        (np.nan, np.nan),
        (-1, -1),
        "no_valid_pixel",
    )

def calculate_statistics(
    primary_data: np.ndarray,
    comparison_data: np.ndarray,
) -> dict:
    """Return RMSE, MAD, and correlation metrics between two series."""

    diff = primary_data - comparison_data
    rmse = np.sqrt(np.nanmean(diff ** 2))
    mad = st.median_abs_deviation(diff, nan_policy='omit')
    try:
        r_val = st.pearsonr(primary_data, comparison_data)[0]
    except (ValueError, ZeroDivisionError):
        r_val = np.nan
    return {'rmse': rmse, 'mad': mad, 'r2': r_val}


def add_stats_box(
    ax,
    stats: dict,
    fontsize: int,
    unit: str = 'cm',
) -> None:
    """Embed a statistics box inside the provided axes."""

    stats_text = (
        f"R2: {stats['r2']:.2f}\n"
        f"RMSE: {stats['rmse']:.2f} {unit}\n"
        f"MAD: {stats['mad']:.2f} {unit}"
    )
    ax.text(
        0.05,
        0.98,
        stats_text,
        transform=ax.transAxes,
        fontsize=fontsize - 2,
        verticalalignment='top',
        horizontalalignment='left',
    )


def compare_ts_detailed(
    primary_dir: str,
    comparison_dir: str,
    ts_filename: str = 'timeseries.h5',
    fig_dir: str = 'figures',
    fig_ind: int = 0,
    pixel_location: Optional[Tuple[int, int]] = None,
    radius: int = 3,
    labels: Optional[Tuple[str, str]] = None,
    colors: Tuple[str, str] = ('blue', 'magenta'),
    scale: float = 100.0,
    unit: str = 'cm',
    fontsize: int = 8,
    ref_change_dates: Optional[Sequence[Union[str, dt]]] = None,
    rng: Optional[np.random.Generator] = None,
    subtract_initial_epoch: bool = True,
    align_to_primary_second_valid: bool = False,
) -> Tuple[plt.Figure, dict, dict]:
    """Compare two time series stacks with summary plots."""

    labels = labels or ('Primary', 'Comparison')
    alignment_note: Optional[str] = None

    (
        dates,
        primary_values,
        comparison_values,
        lat_lon,
        pixel_location,
        selection_note,
    ) = extract_timeseries_data(
        primary_dir,
        comparison_dir,
        ts_filename=ts_filename,
        pixel_location=pixel_location,
        radius=radius,
        rng=rng,
        subtract_initial_epoch=subtract_initial_epoch,
    )

    if align_to_primary_second_valid:
        common_mask = (~np.isnan(primary_values)) & (~np.isnan(comparison_values))
        valid_indices = np.where(common_mask)[0]
        if valid_indices.size >= 2:
            align_idx = int(valid_indices[1])
            shift_value = comparison_values[align_idx] - primary_values[align_idx]
            comparison_values = comparison_values - shift_value
            #alignment_note = (
            #    f"Aligned comparison series by {shift_value:.3f} at index {align_idx}."
            #)
        else:
            print(
                "Skipping perpendicular baseline alignment; insufficient overlapping points."
            )
    subplots_positions = {
        'ts': [0.03, 0.22, 0.73, 0.76],
        'sp': [0.83, 0.33, 0.16, 0.65],
        'hist': [0.83, 0.04, 0.16, 0.15],
        'ts_dif': [0.03, 0.02, 0.73, 0.17],
    }

    plot_primary = primary_values.copy()
    plot_comparison = comparison_values.copy()
    if np.isnan(plot_primary[0]):
        plot_primary[0] = 0
    if np.isnan(plot_comparison[0]):
        plot_comparison[0] = 0

    valid_mask = (
        ~np.isnan(primary_values) & ~np.isnan(comparison_values)
    )
    if np.count_nonzero(valid_mask) < 2:
        stats = {'rmse': np.nan, 'mad': np.nan, 'r2': np.nan}
    else:
        stats = calculate_statistics(
            primary_values[valid_mask] * scale,
            comparison_values[valid_mask] * scale,
        )

    fig = plt.figure(figsize=(18 / 2.54, 6 / 2.54), layout='none', dpi=300)
    axes = {
        'ts': fig.add_axes(subplots_positions['ts']),
        'ts_dif': fig.add_axes(subplots_positions['ts_dif']),
        'sp': fig.add_axes(subplots_positions['sp']),
        'hist': fig.add_axes(subplots_positions['hist']),
    }

    window_size = 2 * radius + 1
    col_idx, row_idx = pixel_location
    title = (
        f"Pixel at Row:{row_idx}, Col:{col_idx}, "
        f"Lat:{lat_lon[0]:.4f}°, Lon:{lat_lon[1]:.4f}°\n"
        f"Point #{int(fig_ind)}, "
        f"{window_size}x{window_size} window average"
    )
    combined_note = selection_note
    if alignment_note:
        combined_note = (
            alignment_note
            if combined_note is None
            else alignment_note + "\n" + combined_note
        )
    if combined_note:
        title = combined_note + "\n" + title

    ts_data = {
        labels[0]: plot_primary,
        labels[1]: plot_comparison,
    }
    for label, color in zip(labels, colors):
        axes['ts'].plot(
            dates,
            ts_data[label] * scale,
            'o',
            color=color,
            label=f"{label.upper()}",
            ms=3,
        )

    y_candidates: List[np.ndarray] = []
    for label in labels:
        scaled_series = ts_data[label] * scale
        finite_mask = np.isfinite(scaled_series)
        nonzero_mask = finite_mask & (np.abs(scaled_series) > 1.0e-9)
        if np.any(nonzero_mask):
            y_candidates.append(scaled_series[nonzero_mask])
        elif np.any(finite_mask):
            y_candidates.append(scaled_series[finite_mask])
    if y_candidates:
        combined = np.concatenate(y_candidates)
        y_min = float(np.nanmin(combined))
        y_max = float(np.nanmax(combined))
        if not np.isfinite(y_min) or not np.isfinite(y_max):
            pass
        else:
            if np.isclose(y_min, y_max):
                span = max(1.0, abs(y_min) * 0.05 + 1.0)
                y_min -= span
                y_max += span
            else:
                padding = 0.05 * (y_max - y_min)
                y_min -= padding
                y_max += padding
            axes['ts'].set_ylim(y_min, y_max)

    if ref_change_dates:
        parsed_dates = []
        for date_value in ref_change_dates:
            if isinstance(date_value, dt):
                parsed_dates.append(date_value)
            else:
                try:
                    parsed_dates.append(
                    dt.strptime(str(date_value), '%Y%m%d')
                )
                except ValueError:
                    print(f"Warning: Invalid date format for {date_value}")
        for mark_date in parsed_dates:
            if dates[0] <= mark_date <= dates[-1]:
                axes['ts'].axvline(
                    x=mark_date,
                    color='gray',
                    linestyle=':',
                    alpha=0.6,
                    linewidth=2,
                )
                axes['ts_dif'].axvline(
                    x=mark_date,
                    color='gray',
                    linestyle=':',
                    alpha=0.6,
                    linewidth=2,
                )

    axes['ts'].set_ylabel(f'{unit}', fontsize=fontsize)
    axes['ts'].set_title(title, fontsize=fontsize)
    axes['ts'].tick_params(labelsize=fontsize, labelbottom=False)
    axes['ts'].axhline(0, color='gray', lw=0.3, linestyle='--')
    axes['ts'].legend(fontsize=fontsize)

    axes['sp'].plot(
        primary_values[valid_mask] * scale,
        comparison_values[valid_mask] * scale,
        '.',
        ms=1,
    )
    axes['sp'].set_xlabel(
        f'{labels[0]} [{unit}]',
        fontsize=fontsize - 2,
        labelpad=1,
    )
    axes['sp'].set_ylabel(
        f'{labels[1]} [{unit}]',
        fontsize=fontsize - 2,
        labelpad=0.5,
    )
    axes['sp'].tick_params(labelsize=fontsize - 2)
    add_stats_box(axes['sp'], stats, fontsize, unit)

    lims = [
        np.min([axes['sp'].get_xlim(), axes['sp'].get_ylim()]),
        np.max([axes['sp'].get_xlim(), axes['sp'].get_ylim()]),
    ]
    axes['sp'].plot(lims, lims, 'k-', alpha=0.5, zorder=0)

    differences = (primary_values - comparison_values) * scale
    plot_differences = differences.copy()
    if np.isnan(plot_differences[0]):
        plot_differences[0] = 0
    valid_diff_for_hist = differences[1:][~np.isnan(differences[1:])]
    axes['hist'].hist(valid_diff_for_hist, bins=30, color='red', alpha=0.5)
    axes['hist'].set_xlabel(
        f'Diff. [{unit}]',
        fontsize=fontsize - 2,
        labelpad=1,
    )
    axes['hist'].set_ylabel('Count', fontsize=fontsize - 2, labelpad=1)
    axes['hist'].axvline(
        np.nanmean(valid_diff_for_hist),
        color='darkred',
        linestyle='--',
        label='Mean',
    )
    axes['hist'].tick_params(labelsize=fontsize - 2)

    mpl_dates = mdates.date2num(dates)
    axes['ts_dif'].bar(
        mpl_dates,
        plot_differences,
        color='red',
        alpha=0.7,
        label=f'{labels[0].upper()}-{labels[1].upper()}',
    )
    max_abs_diff = max(
        abs(np.nanmin(plot_differences)),
        abs(np.nanmax(plot_differences)),
    )
    axes['ts_dif'].set_ylim(-max_abs_diff or 1, max_abs_diff or 1)
    axes['ts_dif'].set_ylabel(f'Diff. [{unit}]', fontsize=fontsize)
    axes['ts_dif'].tick_params(labelsize=fontsize)
    axes['ts_dif'].axhline(0, color='r', lw=0.5, linestyle='--')
    axes['ts_dif'].xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m'))
    axes['ts_dif'].legend(fontsize=fontsize - 2)

    padding = timedelta(days=15)
    axes['ts'].set_xlim([dates[0] - padding, dates[-1] + padding])

    ensure_directory(fig_dir)
    output_name = (
        f"compare_detailed_{labels[0]}_{labels[1]}_"
        f"{str(fig_ind).zfill(2)}.png"
    )
    output_path = os.path.join(fig_dir, output_name)
    fig.savefig(output_path, bbox_inches='tight', dpi=300)

    return fig, axes, {
        'dates': dates,
        'primary_values': primary_values,
        'comparison_values': comparison_values,
        'pixel_location': pixel_location,
        'lat_lon': lat_lon,
        'selection_note': selection_note,
    }





def run_timeseries_comparison_for_points(
    ts_filename: str,
    lat_lon_points: Sequence[Tuple[float, float]],
    metadata: dict,
    parent_dir: str,
    primary_version: str,
    comparison_version: Optional[str],
    frame_num: Union[str, int],
    pixel_radius: int,
    figures_dir: str,
    scale: float,
    unit: str,
    rng_seed: int,
    ref_change_dates: Optional[Sequence[Union[str, dt]]] = None,
    figure_subdir: Optional[str] = None,
    fallback_wkt_name: Optional[str] = None,
    subtract_initial_epoch: bool = True,
) -> List[Tuple[int, float, float]]:
    """Run compare_ts_detailed for each point and report fallbacks."""

    if not comparison_version:
        print(
            "Comparison version not provided; skipping "
            "timeseries comparison."
        )
        return []

    if not lat_lon_points:
        print("No lat/lon points available for comparison.")
        return []

    base_fig_dir = Path(figures_dir)
    if not base_fig_dir.is_absolute():
        base_fig_dir = Path(parent_dir) / base_fig_dir

    target_fig_dir = (
        base_fig_dir / figure_subdir if figure_subdir else base_fig_dir
    )
    target_fig_dir_str = ensure_directory(target_fig_dir)
    target_fig_dir_path = Path(target_fig_dir_str)

    fallback_wkt_points: List[Tuple[float, float]] = []
    fallback_info: List[Tuple[int, float, float]] = []
    fallback_wkt_path: Optional[Path] = None
    if fallback_wkt_name:
        fallback_wkt_path = target_fig_dir_path / fallback_wkt_name

    primary_dir = build_mintpy_output_dir(
        parent_dir,
        primary_version,
        frame_num,
    )
    comparison_dir = build_mintpy_output_dir(
        parent_dir,
        comparison_version,
        frame_num,
    )

    rng = np.random.default_rng(rng_seed)
    align_perpendicular = figure_subdir == "perpendicular_baseline"

    for idx, (lat, lon) in enumerate(lat_lon_points, start=1):
        pixel_location = latlon_to_pixel(lat, lon, metadata)
        display(
            Markdown(
                f"##### Point {idx:02d}: lat={lat:.4f}, lon={lon:.4f}"
            )
        )
        _, _, data = compare_ts_detailed(
            primary_dir=primary_dir,
            comparison_dir=comparison_dir,
            ts_filename=ts_filename,
            fig_dir=target_fig_dir_str,
            fig_ind=idx,
            pixel_location=pixel_location,
            radius=pixel_radius,
            labels=(primary_version, comparison_version),
            colors=('blue', 'magenta'),
            scale=scale,
            unit=unit,
            fontsize=8,
            ref_change_dates=ref_change_dates,
            rng=rng,
            subtract_initial_epoch=subtract_initial_epoch,
            align_to_primary_second_valid=align_perpendicular,
        )

        actual_lat, actual_lon = data['lat_lon']
        if (
            fallback_wkt_path
            and (
                abs(actual_lat - lat) > 1.0e-3
                or abs(actual_lon - lon) > 1.0e-3
            )
        ):
            fallback_wkt_points.append((actual_lat, actual_lon))
            fallback_info.append((idx, actual_lat, actual_lon))

    if fallback_wkt_points and fallback_wkt_path:
        write_lat_lon_wkt(
            fallback_wkt_points,
            fallback_wkt_path,
            merge_existing=True,
        )
        print(
            f"Recorded {len(fallback_wkt_points)} fallback points to "
            f"{fallback_wkt_path}"
        )

    return fallback_info



### 3.2 Core Plotting Routines


In [ ]:
# ============================
# 3.2 Core Plotting Routines
# ============================

def plot_histogram(
    data: np.ndarray,
    data2: Optional[np.ndarray],
    frame_num: Union[str, int],
    version_label: str,
    compare_label: Optional[str],
    out_file: str,
    bins: int,
    data_range: Optional[Tuple[float, float]],
    density: bool,
    title: str,
    xlabel: str,
    ylabel: str,
) -> plt.Figure:
    """Plot histogram (PDF or raw counts), save PNG, and return the figure."""
    fig, ax = plt.subplots()

    ax.hist(
        data,
        bins=bins,
        range=data_range,
        density=density,
        edgecolor="black",
        alpha=0.35,
        label=f"{version_label}",
    )

    if data2 is not None and compare_label is not None:
        ax.hist(
            data2,
            bins=bins,
            range=data_range,
            density=density,
            edgecolor="black",
            alpha=0.35,
            label=f"{compare_label}",
        )

    ax.set_title(title.format(frame=frame_num))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.grid(axis="y", linestyle="--", alpha=0.7)

    fig.savefig(out_file, dpi=300, bbox_inches="tight")
    return fig


### 3.3 Run Both Threshold Configurations


In [ ]:
# ============================
# 3.3 Run Both Threshold Configurations
# ============================

def run_density_plots(
    frame_num: Union[str, int],
    parent_dir: str,
    version_num: str,
    version_num_to_comp: Optional[str],
    threshold_density: Optional[float],
    bins: int,
    figures_dir: Optional[str] = None,
):
    """Run one set of plots for a single threshold."""
    print("=" * 80)
    print(
        f"Processing frame {frame_num}, "
        f"threshold_density={threshold_density}"
    )
    print("=" * 80)

    # Build file paths
    fp = build_file_path(parent_dir, version_num, frame_num)
    fp_comp = (
        build_file_path(parent_dir, version_num_to_comp, frame_num)
        if version_num_to_comp
        else None
    )

    # Load primary data
    data, count = load_density(fp, threshold_density)
    print(f"Version {version_num}: count = {count}")

    # Load comparison data
    data2 = None
    if fp_comp:
        data2, count2 = load_density(fp_comp, threshold_density)
        print(
            f"Version {version_num_to_comp}: count = {count2} "
            f"(net {count2 - count})\n"
        )

    # --- View plots ---
    view_figures = []

    def nested_capture_view(path: Optional[str], suffix: str) -> None:
        if not path:
            return
        fig = _capture_view_figure(f"{path} {suffix}")
        if fig is not None:
            view_figures.append(fig)

    if threshold_density is None:
        nested_capture_view(
            fp,
            "-v 0 1 -c jet --noverbose --zm --notitle --nocbar",
        )
        if fp_comp:
            nested_capture_view(
                fp_comp,
                "-v 0 1 -c jet --noverbose --zm --notitle",
            )
    else:
        fp_recmsk = build_recommended_mask_path(
            parent_dir=parent_dir,
            version=version_num,
            frame_num=frame_num,
            threshold_density=threshold_density,
        )
        nested_capture_view(
            fp_recmsk,
            f"-v 0.9 1.1 --noverbose --zm --notitle --nocbar",
        )
        if fp_comp:
            fp_comp_recmsk = build_recommended_mask_path(
                parent_dir=parent_dir,
                version=version_num_to_comp,
                frame_num=frame_num,
                threshold_density=threshold_density,
            )
            nested_capture_view(
                fp_comp_recmsk,
                f"-v 0.9 1.1 --noverbose --zm --notitle",
            )

    if view_figures:
        _display_figures_side_by_side(*view_figures)
        for fig in view_figures:
            plt.close(fig)

    # --- PDF plot ---
    pdf_path = make_output_name(
        parent_dir=parent_dir,
        prefix="pdf",
        frame_num=frame_num,
        version_num=version_num,
        version_to_comp=version_num_to_comp,
        threshold_density=threshold_density,
        figures_dir=figures_dir,
    )

    data_range = (
        threshold_density if threshold_density is not None else 0.0,
        1.0,
    )

    fig_pdf = plot_histogram(
        data=data,
        data2=data2,
        frame_num=frame_num,
        version_label=version_num,
        compare_label=version_num_to_comp,
        out_file=pdf_path,
        bins=bins,
        data_range=data_range,
        density=True,
        title="Normalized Histogram (Frame {frame})",
        xlabel="Density (0–1)",
        ylabel="Probability Density",
    )

    # --- Raw histogram ---
    hist_path = make_output_name(
        parent_dir=parent_dir,
        prefix="hist",
        frame_num=frame_num,
        version_num=version_num,
        version_to_comp=version_num_to_comp,
        threshold_density=threshold_density,
        figures_dir=figures_dir,
    )

    fig_hist = plot_histogram(
        data=data,
        data2=data2,
        frame_num=frame_num,
        version_label=version_num,
        compare_label=version_num_to_comp,
        out_file=hist_path,
        bins=bins,
        data_range=None,
        density=False,
        title="Pixel Count Histogram (Frame {frame})",
        xlabel="Density bin",
        ylabel="Frequency",
    )

    _display_figures_side_by_side(fig_pdf, fig_hist)
    plt.close(fig_pdf)
    plt.close(fig_hist)


def run_density_plots_both_thresholds(
    frame_num: Union[str, int],
    parent_dir: str,
    version_num: str,
    version_num_to_comp: Optional[str],
    threshold_density: float,
    bins: int,
    figures_dir: Optional[str] = None,
):
    """Run both: unfiltered (None) and filtered (float) thresholds."""
    # Unfiltered
    run_density_plots(
        frame_num,
        parent_dir,
        version_num,
        version_num_to_comp,
        threshold_density=None,
        bins=bins,
        figures_dir=figures_dir,
    )

    # Thresholded
    run_density_plots(
        frame_num,
        parent_dir,
        version_num,
        version_num_to_comp,
        threshold_density=threshold_density,
        bins=bins,
        figures_dir=figures_dir,
    )


### 3.4 Execution Cell


In [ ]:
# ============================
# 3.4 Execution Cell
# ============================

run_density_plots_both_thresholds(
    frame_num=frame_num,
    parent_dir=parent_dir,
    version_num=version_num,
    version_num_to_comp=version_num_to_comp,
    threshold_density=threshold_density,
    bins=bins,
    figures_dir=figures_dir,
)


## 4. Visualizing Layers by Epoch

Visualize MintPy layers for individual epochs to inspect spatial artifacts side-by-side.


### 4.0 Visualize Selected Timeseries Points

Select representative lat/lon points, persist them to WKT, and display their locations on the geometry height map.


In [ ]:
# Prepare MintPy directories and select lat/lon points
primary_mintpy_dir = build_mintpy_output_dir(
    parent_dir=parent_dir,
    version=version_num,
    frame_num=frame_num,
)
if not version_num_to_comp:
    raise ValueError(
        "A comparison version is required for the timeseries suite."
    )
comparison_mintpy_dir = build_mintpy_output_dir(
    parent_dir=parent_dir,
    version=version_num_to_comp,
    frame_num=frame_num,
)

timeseries_path = build_mintpy_output_path(
    parent_dir=parent_dir,
    version=version_num,
    frame_num=frame_num,
    filename=timeseries_filename,
)
primary_metadata = readfile.read_attribute(timeseries_path)

apply_reference_if_requested(
    ref_lalo=ref_lalo,
    timeseries_path=timeseries_path,
    primary_dir=primary_mintpy_dir,
    comparison_dir=comparison_mintpy_dir,
    metadata=primary_metadata,
)


(
    lat_lon_points,
    random_point_count,
    lat_lon_source,
) = resolve_lat_lon_input(list_lat_lon)
if lat_lon_source:
    print(f"Loaded lat/lon points from: {lat_lon_source}")
if random_point_count:
    print(f"Selecting {random_point_count} random points.")

rng = np.random.default_rng(random_seed)
SELECTED_LAT_LON: List[Tuple[float, float]] = []

if lat_lon_points:
    for lat_lon in lat_lon_points:
        pixel_location = latlon_to_pixel(
            lat_lon[0],
            lat_lon[1],
            primary_metadata,
        )
        try:
            (
                _,
                _,
                _,
                lat_lon_result,
                _,
                _,
            ) = extract_timeseries_data(
                primary_dir=primary_mintpy_dir,
                comparison_dir=comparison_mintpy_dir,
                ts_filename=timeseries_filename,
                pixel_location=pixel_location,
                radius=pixel_radius,
                rng=rng,
                allow_random_fallback=False,
            )
        except ValueError as exc:
            warnings.warn(
                f"Discarding lat/lon point ({lat_lon[0]:.5f}, {lat_lon[1]:.5f}): {exc}",
                UserWarning,
            )
            continue
        SELECTED_LAT_LON.append(lat_lon_result)
elif random_point_count:
    failures = 0
    max_failures = max(200, random_point_count * 50)
    while len(SELECTED_LAT_LON) < random_point_count:
        try:
            (
                _,
                _,
                _,
                lat_lon_result,
                _,
                _,
            ) = extract_timeseries_data(
                primary_dir=primary_mintpy_dir,
                comparison_dir=comparison_mintpy_dir,
                ts_filename=timeseries_filename,
                pixel_location=None,
                radius=pixel_radius,
                rng=rng,
                allow_random_fallback=True,
            )
        except ValueError as exc:
            failures += 1
            if failures >= max_failures:
                raise ValueError(
                    f"Unable to find {random_point_count} valid points after {failures} failed attempts."
                ) from exc
            continue
        SELECTED_LAT_LON.append(lat_lon_result)

figures_path = Path(figures_dir)
if not figures_path.is_absolute():
    figures_path = Path(parent_dir) / figures_path
figures_dir = ensure_directory(figures_path)

default_wkt = (
    figures_path / f"{build_frame_tag(frame_num)}_selected_points.wkt"
)
wkt_output = (
    Path(list_lat_lon_wkt_output) if list_lat_lon_wkt_output else default_wkt
)
output_wkt_path = write_lat_lon_wkt(
    SELECTED_LAT_LON,
    wkt_output,
    merge_existing=True,
)
print(f"Persisted {len(SELECTED_LAT_LON)} points to {output_wkt_path}.")

geometry_path = build_mintpy_output_path(
    parent_dir=parent_dir,
    version=version_num,
    frame_num=frame_num,
    filename="geometryGeo.h5",
)
if not os.path.exists(geometry_path):
    raise FileNotFoundError(f"Missing geometry file: {geometry_path}")

GEOMETRY_PATH = geometry_path
height_map_path = figures_path / (
    f"height_points_{build_frame_tag(frame_num)}.png"
)
plot_height_map_with_points(
    geometry_path=GEOMETRY_PATH,
    metadata=primary_metadata,
    points=SELECTED_LAT_LON,
    title=f"Height map with selected points, {build_frame_tag(frame_num)}",
    output_path=height_map_path,
)
print(f"Saved height map to {height_map_path}")

PRIMARY_METADATA = primary_metadata


### 4.1 Recommended Mask

Load the per-epoch recommended masks for each version and render them separately using `capture_view`.


In [ ]:
# Build recommended mask paths for per-epoch visualization
fp_recmsk_ep = build_epoch_recommended_mask_path(
    parent_dir=parent_dir,
    version=version_num,
    frame_num=frame_num,
)

fp_comp_recmsk_ep = None
if version_num_to_comp:
    fp_comp_recmsk_ep = build_epoch_recommended_mask_path(
        parent_dir=parent_dir,
        version=version_num_to_comp,
        frame_num=frame_num,
    )

print(f"Primary recommended mask path: {fp_recmsk_ep}")
if fp_comp_recmsk_ep:
    print(f"Comparison recommended mask path: {fp_comp_recmsk_ep}")
else:
    print("Comparison version not provided; skipping comparison path.")





In [ ]:
display(Markdown(f"#### {version_num}"))
if not os.path.exists(fp_recmsk_ep):
    raise FileNotFoundError(f"Missing recommended mask file: {fp_recmsk_ep}")
capture_view(
    fp_recmsk_ep,
    "-v 0.9 1.1 --noverbose --zm -u m --cbar-label mask",
)


In [ ]:
if version_num_to_comp and fp_comp_recmsk_ep:
    display(Markdown(f"#### {version_num_to_comp}"))
    if not os.path.exists(fp_comp_recmsk_ep):
        raise FileNotFoundError(
            f"Missing comparison recommended mask file: {fp_comp_recmsk_ep}"
        )
    capture_view(
        fp_comp_recmsk_ep,
        "-v 0.9 1.1 --noverbose --zm -u m --cbar-label mask",
    )
else:
    print("No comparison version available for visualization.")


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Recommended Mask",
    primary_path=fp_recmsk_ep,
    comparison_path=fp_comp_recmsk_ep,
    dataset="timeseries",
    scale=1.0,
)


### 4.2 Displacement

Visualize the displacement timeseries for each version using a common color stretch computed from the primary dataset.


In [ ]:
def _choose_disp_comp_range():
    if vmin_disp == 0 and vmax_disp == 0:
        return vmin_disp_comp, vmax_disp_comp
    return vmin_disp, vmax_disp

# Build displacement file paths and compute color range
fp_disp_ep = build_timeseries_path(
    parent_dir=parent_dir,
    version=version_num,
    frame_num=frame_num,
)

fp_comp_disp_ep = None
if version_num_to_comp:
    fp_comp_disp_ep = build_timeseries_path(
        parent_dir=parent_dir,
        version=version_num_to_comp,
        frame_num=frame_num,
    )

if not os.path.exists(fp_disp_ep):
    raise FileNotFoundError(
        f"Missing displacement file: {fp_disp_ep}"
    )
(vmin_disp, vmax_disp) = compute_displacement_range(fp_disp_ep)
vmin_disp_comp = vmin_disp
vmax_disp_comp = vmax_disp
if fp_comp_disp_ep and os.path.exists(fp_comp_disp_ep):
    (vmin_disp_comp, vmax_disp_comp) = compute_displacement_range(fp_comp_disp_ep)
ref_date = compute_common_reference_date(
    fp_disp_ep,
    fp_comp_disp_ep,
)
ref_arg = f"--ref-date {ref_date}" if ref_date else ""
if ref_date:
    print(f"Reference date for visualization: {ref_date}")
else:
    print(
        "Could not determine a common reference date; "
        "proceeding without --ref-date."
    )
print(f"Primary displacement file: {fp_disp_ep}")
print(
    f"Displacement range (cm): vmin={vmin_disp:.3f}, "
    f"vmax={vmax_disp:.3f}"
)
if fp_comp_disp_ep:
    print(
        f"Comparison displacement file: {fp_comp_disp_ep}"
    )
else:
    print(
        "Comparison version not provided; skipping "
        "comparison displacement path."
    )

disp_common_dates_arg = build_common_date_argument(
    fp_disp_ep,
    fp_comp_disp_ep,
)





In [ ]:

display(Markdown(f"#### {version_num} Displacement"))
displacement_view = (
    f"-v {vmin_disp:.3f} {vmax_disp:.3f} "
    f"{format_common_dates_for_view(disp_common_dates_arg)}"
    f"--noverbose --zm {ref_arg}"
)
capture_view(fp_disp_ep, displacement_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_disp_ep:
    display(
        Markdown(
            f"#### {version_num_to_comp} Displacement"
        )
    )
    displacement_comp_view = (
        f"-v {_choose_disp_comp_range()[0]:.3f} {_choose_disp_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(disp_common_dates_arg)}"
        f"--noverbose --zm {ref_arg}"
    )
    capture_view(
        fp_comp_disp_ep,
        displacement_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for displacement "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Displacement",
    primary_path=fp_disp_ep,
    comparison_path=fp_comp_disp_ep,
    dataset="timeseries",
    scale=100.0,
    primary_range=(vmin_disp, vmax_disp),
)


#### 4.2.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for displacement using the selected points.


In [ ]:
reset_common_dates_cache()
fallback_info_disp = run_timeseries_comparison_for_points(
    ts_filename="timeseries.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=100.0,
    unit="cm",
    rng_seed=random_seed,
    figure_subdir="displacement",
    fallback_wkt_name="displacement_fallback_points.wkt",
)

if fallback_info_disp:
    fallback_map_path = (
        Path(figures_dir)
        / "displacement"
        / "displacement_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Displacement fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_disp,
        highlight_color='red',
    )
else:
    print(
        "All requested points were valid for Displacement."
    )



### 4.3 Short Wavelength Displacement

Display the short-wavelength displacement stack using the shared reference epoch when possible.


In [ ]:
def _choose_short_disp_comp_range():
    if short_disp_vmin == 0 and short_disp_vmax == 0:
        return short_disp_vmin_comp, short_disp_vmax_comp
    return short_disp_vmin, short_disp_vmax

# Build short wavelength displacement file paths and compute color range
short_disp_filename = "short_wavelength_displacement.h5"
fp_short_disp, fp_comp_short_disp = build_epoch_layer_paths(
    filename=short_disp_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_short_disp,
    f"{version_num} short wavelength displacement file",
)
if fp_comp_short_disp:
    ensure_file_exists(
        fp_comp_short_disp,
        f"{version_num_to_comp} short wavelength displacement file",
    )
(short_disp_vmin, short_disp_vmax) = compute_displacement_range(fp_short_disp)
short_disp_vmin_comp = short_disp_vmin
short_disp_vmax_comp = short_disp_vmax
if fp_comp_short_disp and os.path.exists(fp_comp_short_disp):
    (short_disp_vmin_comp, short_disp_vmax_comp) = compute_displacement_range(fp_comp_short_disp)
    print(f"Comparison short_disp range: vmin={short_disp_vmin_comp:.3f}, vmax={short_disp_vmax_comp:.3f}")
short_disp_ref_date = compute_common_reference_date(
    fp_short_disp,
    fp_comp_short_disp,
)
short_disp_ref_arg = (
    f"--ref-date {short_disp_ref_date}"
    if short_disp_ref_date
    else ""
)
if short_disp_ref_date:
    print(
        "Reference date for short wavelength displacement "
        f"visualization: {short_disp_ref_date}"
    )
else:
    print(
        "Could not determine a common reference date for "
        "short wavelength displacement; proceeding "
        "without --ref-date."
    )
print(f"Primary short wavelength displacement file: {fp_short_disp}")
print(
    "Short Wavelength Displacement range (cm): "
    f"vmin={short_disp_vmin:.3f}, vmax={short_disp_vmax:.3f}"
)
if fp_comp_short_disp:
    print(
        f"Comparison short wavelength displacement file: "
        f"{fp_comp_short_disp}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "short wavelength displacement path."
    )





In [ ]:
display(
    Markdown(
        f"#### {version_num} Short Wavelength Displacement"
    )
)
short_disp_view = (
    f"-v {short_disp_vmin:.3f} {short_disp_vmax:.3f} "
    f"--noverbose --zm {short_disp_ref_arg}"
)
capture_view(fp_short_disp, short_disp_view.strip())


In [ ]:
if version_num_to_comp and fp_comp_short_disp:
    display(
        Markdown(
            f"#### {version_num_to_comp} Short Wavelength "
            "Displacement"
        )
    )
    short_disp_comp_view = (
        f"-v {_choose_short_disp_comp_range()[0]:.3f} {_choose_short_disp_comp_range()[1]:.3f} "
        f"--noverbose --zm {short_disp_ref_arg}"
    )
    capture_view(
        fp_comp_short_disp,
        short_disp_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for short wavelength "
        "displacement visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Short Wavelength Displacement",
    primary_path=fp_short_disp,
    comparison_path=fp_comp_short_disp,
    dataset="timeseries",
    scale=100.0,
    primary_range=(short_disp_vmin, short_disp_vmax),
)


#### 4.3.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for short wavelength displacement using the selected points.


In [ ]:
reset_common_dates_cache()
fallback_info_short_disp = run_timeseries_comparison_for_points(
    ts_filename="short_wavelength_displacement.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=100.0,
    unit="cm",
    rng_seed=random_seed,
    figure_subdir="short_wavelength_displacement",
    fallback_wkt_name="short_wavelength_displacement_fallback_points.wkt",
)

if fallback_info_short_disp:
    fallback_map_path = (
        Path(figures_dir)
        / "short_wavelength_displacement"
        / "short_wavelength_displacement_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Short Wavelength Displacement fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_short_disp,
        highlight_color='red',
    )
else:
    print(
        "All requested points were valid for Short Wavelength Displacement."
    )



### 4.4 Ionospheric Delay

Render the per-epoch ionospheric delay correction for both versions.


In [ ]:
def _choose_iono_delay_comp_range():
    if iono_delay_vmin == 0 and iono_delay_vmax == 0:
        return iono_delay_vmin_comp, iono_delay_vmax_comp
    return iono_delay_vmin, iono_delay_vmax

# Build ionospheric delay file paths and compute color range
iono_delay_filename = "ionospheric_delay.h5"
fp_iono_delay, fp_comp_iono_delay = build_epoch_layer_paths(
    filename=iono_delay_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_iono_delay,
    f"{version_num} ionospheric delay file",
)
if fp_comp_iono_delay:
    ensure_file_exists(
        fp_comp_iono_delay,
        f"{version_num_to_comp} ionospheric delay file",
    )
(iono_delay_vmin, iono_delay_vmax) = compute_displacement_range(fp_iono_delay)
iono_delay_vmin_comp = iono_delay_vmin
iono_delay_vmax_comp = iono_delay_vmax
if fp_comp_iono_delay and os.path.exists(fp_comp_iono_delay):
    (iono_delay_vmin_comp, iono_delay_vmax_comp) = compute_displacement_range(fp_comp_iono_delay)
    print(f"Comparison iono_delay range: vmin={iono_delay_vmin_comp:.3f}, vmax={iono_delay_vmax_comp:.3f}")
iono_delay_ref_date = compute_common_reference_date(
    fp_iono_delay,
    fp_comp_iono_delay,
)
iono_delay_ref_arg = (
    f"--ref-date {iono_delay_ref_date}"
    if iono_delay_ref_date
    else ""
)
if iono_delay_ref_date:
    print(
        "Reference date for ionospheric delay visualization: "
        f"{iono_delay_ref_date}"
    )
else:
    print(
        "Could not determine a common reference date for "
        "ionospheric delay; proceeding without --ref-date."
    )
print(f"Primary ionospheric delay file: {fp_iono_delay}")
print(
    "Ionospheric Delay range (cm): vmin="
    f"{iono_delay_vmin:.3f}, vmax={iono_delay_vmax:.3f}"
)
if fp_comp_iono_delay:
    print(
        f"Comparison ionospheric delay file: {fp_comp_iono_delay}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "ionospheric delay path."
    )

iono_delay_common_dates_arg = build_common_date_argument(
    fp_iono_delay,
    fp_comp_iono_delay,
)




In [ ]:

display(Markdown(f"#### {version_num} Ionospheric Delay"))
iono_delay_view = (
    f"-v {iono_delay_vmin:.3f} {iono_delay_vmax:.3f} "
    f"{format_common_dates_for_view(iono_delay_common_dates_arg)}"
    f"--noverbose --zm {iono_delay_ref_arg}"
)
capture_view(fp_iono_delay, iono_delay_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_iono_delay:
    display(
        Markdown(
            f"#### {version_num_to_comp} Ionospheric Delay"
        )
    )
    iono_delay_comp_view = (
        f"-v {_choose_iono_delay_comp_range()[0]:.3f} {_choose_iono_delay_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(iono_delay_common_dates_arg)}"
        f"--noverbose --zm {iono_delay_ref_arg}"
    )
    capture_view(
        fp_comp_iono_delay,
        iono_delay_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for ionospheric delay "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Ionospheric Delay",
    primary_path=fp_iono_delay,
    comparison_path=fp_comp_iono_delay,
    dataset="timeseries",
    scale=100.0,
    primary_range=(iono_delay_vmin, iono_delay_vmax),
)


#### 4.4.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for ionospheric delay using the selected points.


In [ ]:
reset_common_dates_cache()
fallback_info_iono = run_timeseries_comparison_for_points(
    ts_filename="ionospheric_delay.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=100.0,
    unit="cm",
    rng_seed=random_seed,
    figure_subdir="ionospheric_delay",
    fallback_wkt_name="ionospheric_delay_fallback_points.wkt",
)

if fallback_info_iono:
    fallback_map_path = (
        Path(figures_dir)
        / "ionospheric_delay"
        / "ionospheric_delay_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Ionospheric Delay fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_iono,
        highlight_color='red',
    )
else:
    print(
        "All requested points were valid for Ionospheric Delay."
    )



### 4.5 Solid Earth Tide

Compare the solid earth tide displacement contribution per epoch.


In [ ]:
def _choose_solid_earth_comp_range():
    if solid_earth_vmin == 0 and solid_earth_vmax == 0:
        return solid_earth_vmin_comp, solid_earth_vmax_comp
    return solid_earth_vmin, solid_earth_vmax

# Build solid earth tide file paths and compute color range
solid_earth_filename = "solid_earth_tide.h5"
fp_solid_earth, fp_comp_solid_earth = build_epoch_layer_paths(
    filename=solid_earth_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_solid_earth,
    f"{version_num} solid earth tide file",
)
if fp_comp_solid_earth:
    ensure_file_exists(
        fp_comp_solid_earth,
        f"{version_num_to_comp} solid earth tide file",
    )
(solid_earth_vmin, solid_earth_vmax) = compute_displacement_range(fp_solid_earth)
solid_earth_vmin_comp = solid_earth_vmin
solid_earth_vmax_comp = solid_earth_vmax
if fp_comp_solid_earth and os.path.exists(fp_comp_solid_earth):
    (solid_earth_vmin_comp, solid_earth_vmax_comp) = compute_displacement_range(fp_comp_solid_earth)
    print(f"Comparison solid_earth range: vmin={solid_earth_vmin_comp:.3f}, vmax={solid_earth_vmax_comp:.3f}")
)
solid_earth_ref_date = compute_common_reference_date(
    fp_solid_earth,
    fp_comp_solid_earth,
)
solid_earth_ref_arg = (
    f"--ref-date {solid_earth_ref_date}"
    if solid_earth_ref_date
    else ""
)
if solid_earth_ref_date:
    print(
        "Reference date for solid earth tide visualization: "
        f"{solid_earth_ref_date}"
    )
else:
    print(
        "Could not determine a common reference date for solid "
        "earth tide; proceeding without --ref-date."
    )
print(f"Primary solid earth tide file: {fp_solid_earth}")
print(
    "Solid Earth Tide range (cm): vmin="
    f"{solid_earth_vmin:.3f}, vmax={solid_earth_vmax:.3f}"
)
if fp_comp_solid_earth:
    print(
        f"Comparison solid earth tide file: {fp_comp_solid_earth}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "solid earth tide path."
    )

solid_earth_common_dates_arg = build_common_date_argument(
    fp_solid_earth,
    fp_comp_solid_earth,
)





In [ ]:

display(Markdown(f"#### {version_num} Solid Earth Tide"))
solid_earth_view = (
    f"-v {solid_earth_vmin:.3f} {solid_earth_vmax:.3f} "
    f"{format_common_dates_for_view(solid_earth_common_dates_arg)}"
    f"--noverbose --zm {solid_earth_ref_arg}"
)
capture_view(fp_solid_earth, solid_earth_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_solid_earth:
    display(
        Markdown(
            f"#### {version_num_to_comp} Solid Earth Tide"
        )
    )
    solid_earth_comp_view = (
        f"-v {_choose_solid_earth_comp_range()[0]:.3f} {_choose_solid_earth_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(solid_earth_common_dates_arg)}"
        f"--noverbose --zm {solid_earth_ref_arg}"
    )
    capture_view(
        fp_comp_solid_earth,
        solid_earth_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for solid earth tide "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Solid Earth Tide",
    primary_path=fp_solid_earth,
    comparison_path=fp_comp_solid_earth,
    dataset="timeseries",
    scale=100.0,
    primary_range=(solid_earth_vmin, solid_earth_vmax),
)


#### 4.5.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for solid earth tide using the selected points.


In [ ]:
reset_common_dates_cache()
fallback_info_solid = run_timeseries_comparison_for_points(
    ts_filename="solid_earth_tide.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=100.0,
    unit="cm",
    rng_seed=random_seed,
    figure_subdir="solid_earth_tide",
    fallback_wkt_name="solid_earth_tide_fallback_points.wkt",
)

if fallback_info_solid:
    fallback_map_path = (
        Path(figures_dir)
        / "solid_earth_tide"
        / "solid_earth_tide_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Solid Earth Tide fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_solid,
        highlight_color='red',
    )
else:
    print(
        "All requested points were valid for Solid Earth Tide."
    )



### 4.6 Perpendicular Baseline

Review the per-epoch perpendicular baseline measurements for each version.


In [ ]:
def _choose_perp_baseline_comp_range():
    if perp_baseline_vmin == 0 and perp_baseline_vmax == 0:
        return perp_baseline_vmin_comp, perp_baseline_vmax_comp
    return perp_baseline_vmin, perp_baseline_vmax


# Build perpendicular baseline file paths and compute color range
perp_baseline_filename = "perpendicular_baseline.h5"
(
    fp_perp_baseline,
    fp_comp_perp_baseline,
) = build_epoch_layer_paths(
    filename=perp_baseline_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_perp_baseline,
    f"{version_num} perpendicular baseline file",
)
if fp_comp_perp_baseline:
    ensure_file_exists(
        fp_comp_perp_baseline,
        f"{version_num_to_comp} perpendicular baseline file",
    )
(perp_baseline_vmin, perp_baseline_vmax) = compute_displacement_range(fp_perp_baseline)
perp_baseline_vmin_comp = perp_baseline_vmin
perp_baseline_vmax_comp = perp_baseline_vmax
if fp_comp_perp_baseline and os.path.exists(fp_comp_perp_baseline):
    (perp_baseline_vmin_comp, perp_baseline_vmax_comp) = compute_displacement_range(fp_comp_perp_baseline)
    print(f"Comparison perp_baseline range: vmin={perp_baseline_vmin_comp:.3f}, vmax={perp_baseline_vmax_comp:.3f}")
perp_baseline_ref_date = compute_common_reference_date(
    fp_perp_baseline,
    fp_comp_perp_baseline,
)
perp_baseline_ref_arg = (
    f"--ref-date {perp_baseline_ref_date}"
    if perp_baseline_ref_date
    else ""
)
if perp_baseline_ref_date:
    print(
        "Reference date for perpendicular baseline visualization: "
        f"{perp_baseline_ref_date}"
    )
else:
    print(
        "Could not determine a common reference date for "
        "perpendicular baseline; proceeding without --ref-date."
    )
print(f"Primary perpendicular baseline file: {fp_perp_baseline}")
print(
    "Perpendicular Baseline range (cm): vmin="
    f"{perp_baseline_vmin:.3f}, vmax={perp_baseline_vmax:.3f}"
)
if fp_comp_perp_baseline:
    print(
        "Comparison perpendicular baseline file: "
        f"{fp_comp_perp_baseline}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "perpendicular baseline path."
    )

perp_baseline_common_dates_arg = build_common_date_argument(
    fp_perp_baseline,
    fp_comp_perp_baseline,
)





In [ ]:

display(Markdown(f"#### {version_num} Perpendicular Baseline"))
perp_baseline_view = (
    f"-v {perp_baseline_vmin:.3f} {perp_baseline_vmax:.3f} "
    f"{format_common_dates_for_view(perp_baseline_common_dates_arg)}"
    f"--noverbose --zm {perp_baseline_ref_arg}"
)
capture_view(fp_perp_baseline, perp_baseline_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_perp_baseline:
    display(Markdown(f"#### {version_num_to_comp} Perpendicular Baseline"))
    perp_baseline_comp_view = (
        f"-v {_choose_perp_baseline_comp_range()[0]:.3f} {_choose_perp_baseline_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(perp_baseline_common_dates_arg)}"
        f"--noverbose --zm {perp_baseline_ref_arg}"
    )
    capture_view(
        fp_comp_perp_baseline,
        perp_baseline_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for perpendicular baseline "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Perpendicular Baseline",
    primary_path=fp_perp_baseline,
    comparison_path=fp_comp_perp_baseline,
    dataset="timeseries",
    scale=100.0,
    primary_range=(perp_baseline_vmin, perp_baseline_vmax),
)


#### 4.6.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for perpendicular baseline using the selected points.


In [ ]:

reset_common_dates_cache()
fallback_info_perp = run_timeseries_comparison_for_points(
    ts_filename="perpendicular_baseline.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=100.0,
    unit="cm",
    rng_seed=random_seed,
    figure_subdir="perpendicular_baseline",
    fallback_wkt_name="perpendicular_baseline_fallback_points.wkt",
)

if fallback_info_perp:
    fallback_map_path = (
        Path(figures_dir)
        / "perpendicular_baseline"
        / "perpendicular_baseline_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Perpendicular Baseline fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_perp,
        highlight_color='red',
    )
else:
    print("All requested points were valid for Perpendicular Baseline.")


### 4.7 Connected Component Labels

Inspect the connected component labels that define consistent phase regions.


In [ ]:
def _choose_conn_comp_comp_range():
    if conn_comp_vmin == 0 and conn_comp_vmax == 0:
        return conn_comp_vmin_comp, conn_comp_vmax_comp
    return conn_comp_vmin, conn_comp_vmax

# Build connected component labels file paths and compute color range
conn_comp_filename = "connected_component_labels.h5"
fp_conn_comp, fp_comp_conn_comp = build_epoch_layer_paths(
    filename=conn_comp_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_conn_comp,
    f"{version_num} connected component label file",
)
if fp_comp_conn_comp:
    ensure_file_exists(
        fp_comp_conn_comp,
        f"{version_num_to_comp} connected component label file",
    )
(conn_comp_vmin, conn_comp_vmax) = compute_displacement_range(fp_conn_comp,
    scale=1.0,)
conn_comp_vmin_comp = conn_comp_vmin
conn_comp_vmax_comp = conn_comp_vmax
if fp_conn_comp and os.path.exists(fp_conn_comp):
    (conn_comp_vmin_comp, conn_comp_vmax_comp) = compute_displacement_range(fp_conn_comp)
    print(f"Comparison conn_comp range: vmin={conn_comp_vmin_comp:.3f}, vmax={conn_comp_vmax_comp:.3f}")
conn_comp_ref_arg = "-u m --cbar-label conn_comp"
print(f"Primary connected component label file: {fp_conn_comp}")
print(
    "Connected Component Labels range (labels): vmin="
    f"{conn_comp_vmin:.3f}, vmax={conn_comp_vmax:.3f}"
)
if fp_comp_conn_comp:
    print(
        f"Comparison connected component label file: "
        f"{fp_comp_conn_comp}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "connected component label path."
    )

conn_comp_common_dates_arg = build_common_date_argument(
    fp_conn_comp,
    fp_comp_conn_comp,
)




In [ ]:

display(
    Markdown(
        f"#### {version_num} Connected Component Labels"
    )
)
conn_comp_view = (
    f"-v {conn_comp_vmin:.3f} {conn_comp_vmax:.3f} "
    f"{format_common_dates_for_view(conn_comp_common_dates_arg)}"
    f"--noverbose --zm {conn_comp_ref_arg}"
)
capture_view(fp_conn_comp, conn_comp_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_conn_comp:
    display(
        Markdown(
            f"#### {version_num_to_comp} Connected Component "
            "Labels"
        )
    )
    conn_comp_comp_view = (
        f"-v {_choose_conn_comp_comp_range()[0]:.3f} {_choose_conn_comp_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(conn_comp_common_dates_arg)}"
        f"--noverbose --zm {conn_comp_ref_arg}"
    )
    capture_view(
        fp_comp_conn_comp,
        conn_comp_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for connected component "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Connected Component Labels",
    primary_path=fp_conn_comp,
    comparison_path=fp_comp_conn_comp,
    dataset="timeseries",
    scale=1.0,
    primary_range=(conn_comp_vmin, conn_comp_vmax),
)


### 4.8 Shape Counts

Display the shape-count layer.


In [ ]:
def _choose_shape_counts_comp_range():
    if shape_counts_vmin == 0 and shape_counts_vmax == 0:
        return shape_counts_vmin_comp, shape_counts_vmax_comp
    return shape_counts_vmin, shape_counts_vmax


# Build shape counts file paths and compute color range
shape_counts_filename = "shp_counts.h5"
fp_shape_counts, fp_comp_shape_counts = build_epoch_layer_paths(
    filename=shape_counts_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_shape_counts,
    f"{version_num} shape counts file",
)
if fp_comp_shape_counts:
    ensure_file_exists(
        fp_comp_shape_counts,
        f"{version_num_to_comp} shape counts file",
    )
(shape_counts_vmin, shape_counts_vmax) = compute_displacement_range(fp_shape_counts,
    scale=1.0,)
shape_counts_vmin_comp = shape_counts_vmin
shape_counts_vmax_comp = shape_counts_vmax
if fp_comp_shape_counts and os.path.exists(fp_comp_shape_counts):
    (shape_counts_vmin_comp, shape_counts_vmax_comp) = compute_displacement_range(fp_comp_shape_counts,
    scale=1.0,)
    print(f"Comparison shape_counts range: vmin={shape_counts_vmin_comp:.3f}, vmax={shape_counts_vmax_comp:.3f}")
shape_counts_ref_arg = "-u m --cbar-label shape_counts"
print(f"Primary shape counts file: {fp_shape_counts}")
print(
    "Shape Counts range: vmin="
    f"{shape_counts_vmin:.3f}, vmax={shape_counts_vmax:.3f}"
)
if fp_comp_shape_counts:
    print(
        f"Comparison shape counts file: {fp_comp_shape_counts}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "shape counts path."
    )

shape_counts_common_dates_arg = build_common_date_argument(
    fp_shape_counts,
    fp_comp_shape_counts,
)


In [ ]:

display(Markdown(f"#### {version_num} Shape Counts"))
shape_counts_view = (
    f"-v {shape_counts_vmin:.3f} {shape_counts_vmax:.3f} "
    f"{format_common_dates_for_view(shape_counts_common_dates_arg)}"
    f"--noverbose --zm {shape_counts_ref_arg}"
)
capture_view(fp_shape_counts, shape_counts_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_shape_counts:
    display(Markdown(f"#### {version_num_to_comp} Shape Counts"))
    shape_counts_comp_view = (
        f"-v {_choose_shape_counts_comp_range()[0]:.3f} {_choose_shape_counts_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(shape_counts_common_dates_arg)}"
        f"--noverbose --zm {shape_counts_ref_arg}"
    )
    capture_view(
        fp_comp_shape_counts,
        shape_counts_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for shape counts "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Shape Counts",
    primary_path=fp_shape_counts,
    comparison_path=fp_comp_shape_counts,
    dataset="timeseries",
    scale=1.0,
    primary_range=(shape_counts_vmin, shape_counts_vmax),
)


#### 4.8.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for shape counts using the selected points.


In [ ]:
reset_common_dates_cache()
fallback_info_shape_counts = run_timeseries_comparison_for_points(
    ts_filename="shp_counts.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=1.0,
    unit="unitless",
    rng_seed=random_seed,
    figure_subdir="shape_counts",
    fallback_wkt_name="shape_counts_fallback_points.wkt",
    subtract_initial_epoch=False,
)

if fallback_info_shape_counts:
    fallback_map_path = (
        Path(figures_dir)
        / "shape_counts"
        / "shape_counts_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Shape Counts fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_shape_counts,
        highlight_color='red',
    )
else:
    print("All requested points were valid for Shape Counts.")


### 4.9 Temporal Coherence

Compare the temporal coherence surfaces to highlight decorrelated areas.


In [ ]:
def _choose_temp_coh_comp_range():
    if temp_coh_vmin == 0 and temp_coh_vmax == 0:
        return temp_coh_vmin_comp, temp_coh_vmax_comp
    return temp_coh_vmin, temp_coh_vmax

# Build temporal coherence file paths and compute color range
temp_coh_filename = "temporal_coherence.h5"
fp_temp_coh, fp_comp_temp_coh = build_epoch_layer_paths(
    filename=temp_coh_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_temp_coh,
    f"{version_num} temporal coherence file",
)
if fp_comp_temp_coh:
    ensure_file_exists(
        fp_comp_temp_coh,
        f"{version_num_to_comp} temporal coherence file",
    )
(temp_coh_vmin, temp_coh_vmax) = compute_displacement_range(fp_temp_coh,
    scale=1.0,)
temp_coh_vmin_comp = temp_coh_vmin
temp_coh_vmax_comp = temp_coh_vmax
if fp_comp_temp_coh and os.path.exists(fp_comp_temp_coh):
    (temp_coh_vmin_comp, temp_coh_vmax_comp) = compute_displacement_range(fp_comp_temp_coh,
    scale=1.0,)
    print(f"Comparison temp_coh range: vmin={temp_coh_vmin_comp:.3f}, vmax={temp_coh_vmax_comp:.3f}")
temp_coh_ref_arg = "-u m --cbar-label temp_coh"
print(f"Primary temporal coherence file: {fp_temp_coh}")
print(
    "Temporal Coherence range: vmin="
    f"{temp_coh_vmin:.3f}, vmax={temp_coh_vmax:.3f}"
)
if fp_comp_temp_coh:
    print(
        f"Comparison temporal coherence file: {fp_comp_temp_coh}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "temporal coherence path."
    )

temp_coh_common_dates_arg = build_common_date_argument(
    fp_temp_coh,
    fp_comp_temp_coh,
)




In [ ]:

display(Markdown(f"#### {version_num} Temporal Coherence"))
temp_coh_view = (
    f"-v {temp_coh_vmin:.3f} {temp_coh_vmax:.3f} "
    f"{format_common_dates_for_view(temp_coh_common_dates_arg)}"
    f"--noverbose --zm {temp_coh_ref_arg}"
)
capture_view(fp_temp_coh, temp_coh_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_temp_coh:
    display(
        Markdown(
            f"#### {version_num_to_comp} Temporal Coherence"
        )
    )
    temp_coh_comp_view = (
        f"-v {_choose_temp_coh_comp_range()[0]:.3f} {_choose_temp_coh_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(temp_coh_common_dates_arg)}"
        f"--noverbose --zm {temp_coh_ref_arg}"
    )
    capture_view(
        fp_comp_temp_coh,
        temp_coh_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for temporal coherence "
        "visualization."
    )


report_combined_range(
    description="Temporal Coherence",
    primary_path=fp_temp_coh,
    comparison_path=fp_comp_temp_coh,
    dataset="timeseries",
    scale=1.0,
    primary_range=(temp_coh_vmin, temp_coh_vmax),
)



#### 4.9.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for temporal coherence using the selected points.


In [ ]:
reset_common_dates_cache()
fallback_info_temp_coh = run_timeseries_comparison_for_points(
    ts_filename="temporal_coherence.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=1.0,
    unit="unitless",
    rng_seed=random_seed,
    figure_subdir="temporal_coherence",
    fallback_wkt_name="temporal_coherence_fallback_points.wkt",
    subtract_initial_epoch=False,
)

if fallback_info_temp_coh:
    fallback_map_path = (
        Path(figures_dir)
        / "temporal_coherence"
        / "temporal_coherence_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Temporal Coherence fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_temp_coh,
        highlight_color='red',
    )
else:
    print(
        "All requested points were valid for Temporal Coherence."
    )



### 4.10 Estimated Phase Quality

Visualize the estimated phase quality metric per epoch.


In [ ]:
def _choose_phase_quality_comp_range():
    if phase_quality_vmin == 0 and phase_quality_vmax == 0:
        return phase_quality_vmin_comp, phase_quality_vmax_comp
    return phase_quality_vmin, phase_quality_vmax

# Build estimated phase quality file paths and compute color range
phase_quality_filename = "estimated_phase_quality.h5"
fp_phase_quality, fp_comp_phase_quality = build_epoch_layer_paths(
    filename=phase_quality_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_phase_quality,
    f"{version_num} estimated phase quality file",
)
if fp_comp_phase_quality:
    ensure_file_exists(
        fp_comp_phase_quality,
        f"{version_num_to_comp} estimated phase quality file",
    )
(phase_quality_vmin, phase_quality_vmax) = compute_displacement_range(fp_phase_quality,
    scale=1.0,)
phase_quality_vmin_comp = phase_quality_vmin
phase_quality_vmax_comp = phase_quality_vmax
if fp_comp_phase_quality and os.path.exists(fp_comp_phase_quality):
    (phase_quality_vmin_comp, phase_quality_vmax_comp) = compute_displacement_range(fp_comp_phase_quality,
    scale=1.0,)
    print(f"Comparison phase_quality range: vmin={phase_quality_vmin_comp:.3f}, vmax={phase_quality_vmax_comp:.3f}")
phase_quality_ref_arg = "-u m --cbar-label phs_quality"
print(f"Primary estimated phase quality file: {fp_phase_quality}")
print(
    "Estimated Phase Quality range: vmin="
    f"{phase_quality_vmin:.3f}, vmax={phase_quality_vmax:.3f}"
)
if fp_comp_phase_quality:
    print(
        "Comparison estimated phase quality file: "
        f"{fp_comp_phase_quality}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "estimated phase quality path."
    )

phase_quality_common_dates_arg = build_common_date_argument(
    fp_phase_quality,
    fp_comp_phase_quality,
)




In [ ]:

display(
    Markdown(
        f"#### {version_num} Estimated Phase Quality"
    )
)
phase_quality_view = (
    f"-v {phase_quality_vmin:.3f} {phase_quality_vmax:.3f} "
    f"{format_common_dates_for_view(phase_quality_common_dates_arg)}"
    f"--noverbose --zm {phase_quality_ref_arg}"
)
capture_view(fp_phase_quality, phase_quality_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_phase_quality:
    display(
        Markdown(
            f"#### {version_num_to_comp} Estimated Phase "
            "Quality"
        )
    )
    phase_quality_comp_view = (
        f"-v {_choose_phase_quality_comp_range()[0]:.3f} {_choose_phase_quality_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(phase_quality_common_dates_arg)}"
        f"--noverbose --zm {phase_quality_ref_arg}"
    )
    capture_view(
        fp_comp_phase_quality,
        phase_quality_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for estimated phase "
        "quality visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Estimated Phase Quality",
    primary_path=fp_phase_quality,
    comparison_path=fp_comp_phase_quality,
    dataset="timeseries",
    scale=1.0,
    primary_range=(phase_quality_vmin, phase_quality_vmax),
)


#### 4.10.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for estimated phase quality using the selected points.


In [ ]:
reset_common_dates_cache()
fallback_info_phase_quality = run_timeseries_comparison_for_points(
    ts_filename="estimated_phase_quality.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=1.0,
    unit="unitless",
    rng_seed=random_seed,
    figure_subdir="estimated_phase_quality",
    fallback_wkt_name="estimated_phase_quality_fallback_points.wkt",
    subtract_initial_epoch=False,
)

if fallback_info_phase_quality:
    fallback_map_path = (
        Path(figures_dir)
        / "estimated_phase_quality"
        / "estimated_phase_quality_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Estimated Phase Quality fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_phase_quality,
        highlight_color='red',
    )
else:
    print(
        "All requested points were valid for Estimated Phase Quality."
    )



### 4.11 Phase Similarity

Visualize the phase similarity metric per epoch for both versions.


In [ ]:
def _choose_phase_similarity_comp_range():
    if phase_similarity_vmin == 0 and phase_similarity_vmax == 0:
        return phase_similarity_vmin_comp, phase_similarity_vmax_comp
    return phase_similarity_vmin, phase_similarity_vmax

# Build phase similarity file paths and compute color range
phase_similarity_filename = "phase_similarity.h5"
(
    fp_phase_similarity,
    fp_comp_phase_similarity,
) = build_epoch_layer_paths(
    filename=phase_similarity_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_phase_similarity,
    f"{version_num} phase similarity file",
)
if fp_comp_phase_similarity:
    ensure_file_exists(
        fp_comp_phase_similarity,
        f"{version_num_to_comp} phase similarity file",
    )
(phase_similarity_vmin, phase_similarity_vmax) = compute_displacement_range(fp_phase_similarity,
    scale=1.0,)
phase_similarity_vmin_comp = phase_similarity_vmin
phase_similarity_vmax_comp = phase_similarity_vmax
if fp_comp_phase_similarity and os.path.exists(fp_comp_phase_similarity):
    (phase_similarity_vmin_comp, phase_similarity_vmax_comp) = compute_displacement_range(fp_comp_phase_similarity,
    scale=1.0,)
    print(f"Comparison phase_similarity range: vmin={phase_similarity_vmin_comp:.3f}, vmax={phase_similarity_vmax_comp:.3f}")
phase_similarity_ref_date = compute_common_reference_date(
    fp_phase_similarity,
    fp_comp_phase_similarity,
)
phase_similarity_ref_arg = (
    f"--ref-date {phase_similarity_ref_date} --cbar-label phs_similarity"
    if phase_similarity_ref_date
    else "--cbar-label phs_similarity"
)
if phase_similarity_ref_date:
    print(
        "Reference date for phase similarity visualization: "
        f"{phase_similarity_ref_date}"
    )
else:
    print(
        "Could not determine a common reference date for phase similarity; "
        "proceeding without --ref-date."
    )
print(f"Primary phase similarity file: {fp_phase_similarity}")
print(
    "Phase Similarity range: vmin="
    f"{phase_similarity_vmin:.3f}, vmax={phase_similarity_vmax:.3f}"
)
if fp_comp_phase_similarity:
    print(
        f"Comparison phase similarity file: {fp_comp_phase_similarity}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "phase similarity path."
    )

phase_similarity_common_dates_arg = build_common_date_argument(
    fp_phase_similarity,
    fp_comp_phase_similarity,
)






In [ ]:

display(Markdown(f"#### {version_num} Phase Similarity"))
phase_similarity_view = (
    f"-v {phase_similarity_vmin:.3f} {phase_similarity_vmax:.3f} "
    f"{format_common_dates_for_view(phase_similarity_common_dates_arg)}"
    f"--noverbose --zm {phase_similarity_ref_arg}"
)
capture_view(fp_phase_similarity, phase_similarity_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_phase_similarity:
    display(Markdown(f"#### {version_num_to_comp} Phase Similarity"))
    phase_similarity_comp_view = (
        f"-v {_choose_phase_similarity_comp_range()[0]:.3f} {_choose_phase_similarity_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(phase_similarity_common_dates_arg)}"
        f"--noverbose --zm {phase_similarity_ref_arg}"
    )
    capture_view(
        fp_comp_phase_similarity,
        phase_similarity_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for phase similarity "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Phase Similarity",
    primary_path=fp_phase_similarity,
    comparison_path=fp_comp_phase_similarity,
    dataset="timeseries",
    scale=1.0,
    primary_range=(phase_similarity_vmin, phase_similarity_vmax),
)


#### 4.11.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for phase similarity using the selected points.


In [ ]:

reset_common_dates_cache()
fallback_info_phase_similarity = run_timeseries_comparison_for_points(
    ts_filename="phase_similarity.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=1.0,
    unit="unitless",
    rng_seed=random_seed,
    figure_subdir="phase_similarity",
    fallback_wkt_name="phase_similarity_fallback_points.wkt",
    subtract_initial_epoch=False,
)

if fallback_info_phase_similarity:
    fallback_map_path = (
        Path(figures_dir)
        / "phase_similarity"
        / "phase_similarity_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Phase Similarity fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_phase_similarity,
        highlight_color='red',
    )
else:
    print("All requested points were valid for Phase Similarity.")


### 4.12 Water Mask

Render the water mask used to exclude unreliable pixels.


In [ ]:
def _choose_water_mask_comp_range():
    if water_mask_vmin == 0 and water_mask_vmax == 0:
        return water_mask_vmin_comp, water_mask_vmax_comp
    return water_mask_vmin, water_mask_vmax

# Build water mask file paths and compute color range
water_mask_filename = "water_mask.h5"
fp_water_mask, fp_comp_water_mask = build_epoch_layer_paths(
    filename=water_mask_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_water_mask,
    f"{version_num} water mask file",
)
if fp_comp_water_mask:
    ensure_file_exists(
        fp_comp_water_mask,
        f"{version_num_to_comp} water mask file",
    )
(water_mask_vmin, water_mask_vmax) = compute_displacement_range(fp_water_mask,
    scale=1.0,)
water_mask_vmin_comp = water_mask_vmin
water_mask_vmax_comp = water_mask_vmax
if fp_comp_water_mask and os.path.exists(fp_comp_water_mask):
    (water_mask_vmin_comp, water_mask_vmax_comp) = compute_displacement_range(fp_comp_water_mask,
    scale=1.0,)
    print(f"Comparison water_mask range: vmin={water_mask_vmin_comp:.3f}, vmax={water_mask_vmax_comp:.3f}")
water_mask_ref_arg = "-u m --cbar-label mask"
print(f"Primary water mask file: {fp_water_mask}")
print(
    "Water Mask range: vmin="
    f"{water_mask_vmin:.3f}, vmax={water_mask_vmax:.3f}"
)
if fp_comp_water_mask:
    print(f"Comparison water mask file: {fp_comp_water_mask}")
else:
    print(
        "Comparison version not provided; skipping comparison "
        "water mask path."
    )

water_mask_common_dates_arg = build_common_date_argument(
    fp_water_mask,
    fp_comp_water_mask,
)




In [ ]:

display(Markdown(f"#### {version_num} Water Mask"))
water_mask_view = (
    f"-v 0.9 1.1 "
    f"{format_common_dates_for_view(water_mask_common_dates_arg)}"
    f"--noverbose --zm {water_mask_ref_arg}"
)
capture_view(fp_water_mask, water_mask_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_water_mask:
    display(Markdown(f"#### {version_num_to_comp} Water Mask"))
    water_mask_comp_view = (
        f"-v 0.9 1.1 "
        f"{format_common_dates_for_view(water_mask_common_dates_arg)}"
        f"--noverbose --zm {water_mask_ref_arg}"
    )
    capture_view(
        fp_comp_water_mask,
        water_mask_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for water mask "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Water Mask",
    primary_path=fp_water_mask,
    comparison_path=fp_comp_water_mask,
    dataset="timeseries",
    scale=1.0,
    primary_range=(water_mask_vmin, water_mask_vmax),
)


### 4.13 Timeseries Inversion Residuals

Display the per-epoch inversion residuals to spot modeling issues.


In [ ]:
def _choose_ts_inv_resid_comp_range():
    if ts_inv_resid_vmin == 0 and ts_inv_resid_vmax == 0:
        return ts_inv_resid_vmin_comp, ts_inv_resid_vmax_comp
    return ts_inv_resid_vmin, ts_inv_resid_vmax

# Build timeseries inversion residuals file paths and compute color range
ts_inv_resid_filename = "timeseries_inversion_residuals.h5"
fp_ts_inv_resid, fp_comp_ts_inv_resid = build_epoch_layer_paths(
    filename=ts_inv_resid_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_ts_inv_resid,
    f"{version_num} timeseries inversion residual file",
)
if fp_comp_ts_inv_resid:
    ensure_file_exists(
        fp_comp_ts_inv_resid,
        f"{version_num_to_comp} timeseries inversion residual file",
    )
(ts_inv_resid_vmin, ts_inv_resid_vmax) = compute_displacement_range(fp_ts_inv_resid)
ts_inv_resid_vmin_comp = ts_inv_resid_vmin
ts_inv_resid_vmax_comp = ts_inv_resid_vmax
if fp_comp_ts_inv_resid and os.path.exists(fp_comp_ts_inv_resid):
    (ts_inv_resid_vmin_comp, ts_inv_resid_vmax_comp) = compute_displacement_range(fp_comp_ts_inv_resid)
    print(f"Comparison ts_inv_resid range: vmin={ts_inv_resid_vmin_comp:.3f}, vmax={ts_inv_resid_vmax_comp:.3f}")
ts_inv_ref_arg = ""
print(
    f"Primary timeseries inversion residual file: "
    f"{fp_ts_inv_resid}"
)
print(
    "Timeseries Inversion Residual range (cm): vmin="
    f"{ts_inv_resid_vmin:.3f}, vmax={ts_inv_resid_vmax:.3f}"
)
if fp_comp_ts_inv_resid:
    print(
        "Comparison timeseries inversion residual file: "
        f"{fp_comp_ts_inv_resid}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "timeseries inversion residual path."
    )

ts_inv_common_dates_arg = build_common_date_argument(
    fp_ts_inv_resid,
    fp_comp_ts_inv_resid,
)





In [ ]:

display(
    Markdown(
        f"#### {version_num} Timeseries Inversion Residuals"
    )
)
ts_inv_view = (
    f"-v {ts_inv_resid_vmin:.3f} {ts_inv_resid_vmax:.3f} "
    f"{format_common_dates_for_view(ts_inv_common_dates_arg)}"
    f"--noverbose --zm {ts_inv_ref_arg}"
)
capture_view(fp_ts_inv_resid, ts_inv_view.strip())


In [ ]:

if version_num_to_comp and fp_comp_ts_inv_resid:
    display(
        Markdown(
            f"#### {version_num_to_comp} Timeseries "
            "Inversion Residuals"
        )
    )
    ts_inv_comp_view = (
        f"-v {_choose_ts_inv_resid_comp_range()[0]:.3f} {_choose_ts_inv_resid_comp_range()[1]:.3f} "
        f"{format_common_dates_for_view(ts_inv_common_dates_arg)}"
        f"--noverbose --zm {ts_inv_ref_arg}"
    )
    capture_view(
        fp_comp_ts_inv_resid,
        ts_inv_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for timeseries "
        "inversion residual visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Timeseries Inversion Residuals",
    primary_path=fp_ts_inv_resid,
    comparison_path=fp_comp_ts_inv_resid,
    dataset="timeseries",
    scale=100.0,
    primary_range=(ts_inv_resid_vmin, ts_inv_resid_vmax),
)


#### 4.13.i Timeseries Comparisons

Run the detailed timeseries comparison workflow for timeseries inversion residuals using the selected points.


In [ ]:
reset_common_dates_cache()
fallback_info_inv_resid = run_timeseries_comparison_for_points(
    ts_filename="timeseries_inversion_residuals.h5",
    lat_lon_points=SELECTED_LAT_LON,
    metadata=PRIMARY_METADATA,
    parent_dir=parent_dir,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
    frame_num=frame_num,
    pixel_radius=pixel_radius,
    figures_dir=figures_dir,
    scale=100.0,
    unit="cm",
    rng_seed=random_seed,
    figure_subdir="timeseries_inversion_residuals",
    fallback_wkt_name="timeseries_inversion_residuals_fallback_points.wkt",
)

if fallback_info_inv_resid:
    fallback_map_path = (
        Path(figures_dir)
        / "timeseries_inversion_residuals"
        / "timeseries_inversion_residuals_fallback_height.png"
    )
    plot_height_map_with_points(
        geometry_path=GEOMETRY_PATH,
        metadata=PRIMARY_METADATA,
        points=SELECTED_LAT_LON,
        title="Timeseries Inversion Residuals fallback selections",
        output_path=fallback_map_path,
        highlight_info=fallback_info_inv_resid,
        highlight_color='red',
    )
else:
    print(
        "All requested points were valid for Timeseries Inversion Residuals."
    )



### 4.14 DEM Error

Plot the DEM error layer using centimeter units for a consistent stretch.


In [ ]:
def _choose_dem_err_comp_range():
    if dem_err_vmin == 0 and dem_err_vmax == 0:
        return dem_err_vmin_comp, dem_err_vmax_comp
    return dem_err_vmin, dem_err_vmax

# Build dem error file paths and compute color range
dem_err_filename = "demErr.h5"
fp_dem_err, fp_comp_dem_err = build_epoch_layer_paths(
    filename=dem_err_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_dem_err,
    f"{version_num} DEM error file",
)
if fp_comp_dem_err:
    ensure_file_exists(
        fp_comp_dem_err,
        f"{version_num_to_comp} DEM error file",
    )
(dem_err_vmin, dem_err_vmax) = compute_displacement_range(fp_dem_err,
    dataset="dem",)
dem_err_vmin_comp = dem_err_vmin
dem_err_vmax_comp = dem_err_vmax
if fp_comp_dem_err and os.path.exists(fp_comp_dem_err):
    (dem_err_vmin_comp, dem_err_vmax_comp) = compute_displacement_range(fp_comp_dem_err,
    dataset="dem",)
    print(f"Comparison dem_err range: vmin={dem_err_vmin_comp:.3f}, vmax={dem_err_vmax_comp:.3f}")
dem_err_ref_arg = ""
print(f"Primary DEM error file: {fp_dem_err}")
print(
    f"DEM Error range (cm): vmin={dem_err_vmin:.3f}, "
    f"vmax={dem_err_vmax:.3f}"
)
if fp_comp_dem_err:
    print(f"Comparison DEM error file: {fp_comp_dem_err}")
else:
    print(
        "Comparison version not provided; skipping comparison DEM "
        "error path."
    )




In [ ]:
display(Markdown(f"#### {version_num} DEM Error"))
dem_err_view = (
    f"-v {dem_err_vmin:.3f} {dem_err_vmax:.3f} "
    f"--noverbose --zm {dem_err_ref_arg} -u cm"
)
capture_view(fp_dem_err, dem_err_view.strip())


In [ ]:
if version_num_to_comp and fp_comp_dem_err:
    display(Markdown(f"#### {version_num_to_comp} DEM Error"))
    dem_err_comp_view = (
        f"-v {_choose_dem_err_comp_range()[0]:.3f} {_choose_dem_err_comp_range()[1]:.3f} "
        f"--noverbose --zm {dem_err_ref_arg} -u cm"
    )
    capture_view(
        fp_comp_dem_err,
        dem_err_comp_view.strip(),
    )
else:
    print(
        "No comparison version available for dem error "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="DEM Error",
    primary_path=fp_dem_err,
    comparison_path=fp_comp_dem_err,
    dataset="dem",
    scale=100.0,
    primary_range=(dem_err_vmin, dem_err_vmax),
)


## 5. Velocity Layers

Visualize velocity products derived from the MintPy stacks using consistent color stretches for comparable datasets.


### 5.1 Raw Velocity

Render the raw MintPy velocity estimate and store the common stretch for reuse.


In [ ]:
def _choose_velocity_comp_range():
    if velocity_vmin == 0 and velocity_vmax == 0:
        return velocity_vmin_comp, velocity_vmax_comp
    return velocity_vmin, velocity_vmax


# Build raw velocity file paths and compute color range
ts_primary_velocity = build_timeseries_path(
    parent_dir=parent_dir,
    version=version_num,
    frame_num=frame_num,
)
ts_comparison_velocity = None
if version_num_to_comp:
    ts_comparison_velocity = build_timeseries_path(
        parent_dir=parent_dir,
        version=version_num_to_comp,
        frame_num=frame_num,
    )

ensure_file_exists(
    ts_primary_velocity,
    f"{version_num} timeseries file for velocity fitting",
)
if ts_comparison_velocity:
    ensure_file_exists(
        ts_comparison_velocity,
        f"{version_num_to_comp} timeseries file for velocity fitting",
    )

velocity_common_dates = get_common_date_list(
    ts_primary_velocity,
    ts_comparison_velocity,
)
if velocity_common_dates:
    print(
        "Velocity common dates: "
        f"{velocity_common_dates[0]} to {velocity_common_dates[-1]} "
        f"({len(velocity_common_dates)})"
    )
else:
    print(
        "No comparison timeseries provided; using the full primary "
        "timeline for velocity fitting."
    )

fp_velocity = compute_velocity_from_timeseries(
    timeseries_path=ts_primary_velocity,
    selected_dates=velocity_common_dates,
    output_filename="velocity_commondate.h5",
    description=f"{version_num} common-date velocity",
)
fp_comp_velocity = None
if ts_comparison_velocity:
    fp_comp_velocity = compute_velocity_from_timeseries(
        timeseries_path=ts_comparison_velocity,
        selected_dates=velocity_common_dates,
        output_filename="velocity_commondate.h5",
        description=f"{version_num_to_comp} common-date velocity",
    )

(velocity_vmin, velocity_vmax) = compute_displacement_range(fp_velocity,
    dataset="velocity",)
velocity_vmin_comp = velocity_vmin
velocity_vmax_comp = velocity_vmax
if fp_comp_velocity and os.path.exists(fp_comp_velocity):
    (velocity_vmin_comp, velocity_vmax_comp) = compute_displacement_range(fp_comp_velocity,
    dataset="velocity",)
    print(f"Comparison velocity range: vmin={velocity_vmin_comp:.3f}, vmax={velocity_vmax_comp:.3f}")
print(f"Primary common-date velocity file: {fp_velocity}")
print(
    f"Raw Velocity range (cm/year): vmin={velocity_vmin:.3f}, "
    f"vmax={velocity_vmax:.3f}"
)
if fp_comp_velocity:
    print(f"Comparison common-date velocity file: {fp_comp_velocity}")
else:
    print(
        "Comparison version not provided; skipping comparison raw "
        "velocity path."
    )
velocity_common_range = (velocity_vmin, velocity_vmax)
print("Stored velocity_common_range for reuse across Section 5.")




In [ ]:
display(Markdown(f"#### {version_num} Raw Velocity"))
velocity_view = (
    f"-v {velocity_vmin:.3f} {velocity_vmax:.3f} "
    "--noverbose --zm -u cm/year"
)
capture_view(fp_velocity, velocity_view)


In [ ]:
if version_num_to_comp and fp_comp_velocity:
    display(Markdown(f"#### {version_num_to_comp} Raw Velocity"))
    velocity_comp_view = (
        f"-v {_choose_velocity_comp_range()[0]:.3f} {_choose_velocity_comp_range()[1]:.3f} "
        "--noverbose --zm -u cm/year"
    )
    capture_view(fp_comp_velocity, velocity_comp_view)
else:
    print(
        "No comparison version available for raw velocity "
        "visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Raw Velocity",
    primary_path=fp_velocity,
    comparison_path=fp_comp_velocity,
    dataset="velocity",
    scale=100.0,
    primary_range=(velocity_vmin, velocity_vmax),
)


In [ ]:

if version_num_to_comp and fp_comp_velocity:
    diff_figures_dir = Path(ensure_directory(Path(figures_dir) / "differences"))
    diff_outname = diff_figures_dir / Path(fp_comp_velocity).name
    if not diff_outname.exists():
        scp_args = f"{fp_velocity} {fp_comp_velocity} -o {diff_outname}"
        diff.main(scp_args.split())
    display(Markdown("##### Raw Velocity Difference"))
    capture_view(str(diff_outname), velocity_view)
else:
    print(
        "Skipping raw velocity difference; comparison velocity file "
        "is unavailable."
    )


### 5.2 DEM Error Corrected Velocity

Visualize the DEM-error-corrected velocity using the shared stretch from the raw velocity section.


In [ ]:

# Build dem error corrected velocity file paths and compute color range
ts_primary_velocity_demerr = build_mintpy_output_path(
    parent_dir=parent_dir,
    version=version_num,
    frame_num=frame_num,
    filename="timeseries_demErr.h5",
)
ts_comparison_velocity_demerr = None
if version_num_to_comp:
    ts_comparison_velocity_demerr = build_mintpy_output_path(
        parent_dir=parent_dir,
        version=version_num_to_comp,
        frame_num=frame_num,
        filename="timeseries_demErr.h5",
    )

ensure_file_exists(
    ts_primary_velocity_demerr,
    f"{version_num} DEM error corrected timeseries",
)
if ts_comparison_velocity_demerr:
    ensure_file_exists(
        ts_comparison_velocity_demerr,
        f"{version_num_to_comp} DEM error corrected timeseries",
    )

demerr_common_dates = get_common_date_list(
    ts_primary_velocity_demerr,
    ts_comparison_velocity_demerr,
)
if demerr_common_dates:
    print(
        "DEM error corrected common dates: "
        f"{demerr_common_dates[0]} to {demerr_common_dates[-1]} "
        f"({len(demerr_common_dates)})"
    )
else:
    print(
        "No comparison DEM error corrected timeseries; using the "
        "full primary timeline."
    )

fp_velocity_demerr = compute_velocity_from_timeseries(
    timeseries_path=ts_primary_velocity_demerr,
    selected_dates=demerr_common_dates,
    output_filename="velocity_commondate_demErr.h5",
    description=(
        f"{version_num} DEM error corrected common-date velocity"
    ),
)
fp_comp_velocity_demerr = None
if ts_comparison_velocity_demerr:
    fp_comp_velocity_demerr = compute_velocity_from_timeseries(
        timeseries_path=ts_comparison_velocity_demerr,
        selected_dates=demerr_common_dates,
        output_filename="velocity_commondate_demErr.h5",
        description=(
            f"{version_num_to_comp} DEM error corrected "
            "common-date velocity"
        ),
    )

if 'velocity_common_range' not in globals() or velocity_common_range is None:
    raise RuntimeError(
        "velocity_common_range is undefined; run Section 5.1 Raw "
        "Velocity to set the shared stretch."
    )
(
    velocity_demerr_vmin,
    velocity_demerr_vmax,
) = velocity_common_range
print(
    f"Primary DEM error corrected velocity file: {fp_velocity_demerr}"
)
if fp_comp_velocity_demerr:
    print(
        "Comparison DEM error corrected velocity file: "
        f"{fp_comp_velocity_demerr}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "DEM error corrected velocity path."
    )
print(
    "Using shared raw/DEM velocity range (cm/year): vmin="
    f"{velocity_demerr_vmin:.3f}, vmax={velocity_demerr_vmax:.3f}"
)





In [ ]:
display(
    Markdown(
        f"#### {version_num} DEM Error Corrected Velocity"
    )
)
velocity_demerr_view = (
    f"-v {velocity_demerr_vmin:.3f} {velocity_demerr_vmax:.3f} "
    "--noverbose --zm -u cm/year"
)
capture_view(fp_velocity_demerr, velocity_demerr_view)


In [ ]:
if version_num_to_comp and fp_comp_velocity_demerr:
    display(
        Markdown(
            f"#### {version_num_to_comp} DEM Error Corrected "
            "Velocity"
        )
    )
    velocity_demerr_comp_view = (
        f"-v {_choose_velocity_demerr_comp_range()[0]:.3f} {_choose_velocity_demerr_comp_range()[1]:.3f} "
        "--noverbose --zm -u cm/year"
    )
    capture_view(
        fp_comp_velocity_demerr,
        velocity_demerr_comp_view,
    )
else:
    print(
        "No comparison version available for dem error corrected "
        "velocity visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="DEM Error Corrected Velocity",
    primary_path=fp_velocity_demerr,
    comparison_path=fp_comp_velocity_demerr,
    dataset="velocity",
    scale=100.0,
    primary_range=(velocity_demerr_vmin, velocity_demerr_vmax),
)


In [ ]:

if version_num_to_comp and fp_comp_velocity_demerr:
    diff_figures_dir = Path(ensure_directory(Path(figures_dir) / "differences"))
    diff_outname = diff_figures_dir / Path(fp_comp_velocity_demerr).name
    if not diff_outname.exists():
        scp_args = f"{fp_velocity_demerr} {fp_comp_velocity_demerr} -o {diff_outname}"
        diff.main(scp_args.split())
    display(Markdown("##### DEM Error Corrected Velocity Difference"))
    capture_view(str(diff_outname), velocity_demerr_view)
else:
    print(
        "Skipping DEM error corrected velocity difference; "
        "comparison file is unavailable."
    )


### 5.3 Short Wavelength Velocity

Plot the short-wavelength filtered velocity using its own dynamic range.


In [ ]:
def _choose_velocity_shortwvl_comp_range():
    if velocity_shortwvl_vmin == 0 and velocity_shortwvl_vmax == 0:
        return velocity_shortwvl_vmin_comp, velocity_shortwvl_vmax_comp
    return velocity_shortwvl_vmin, velocity_shortwvl_vmax

# Build short wavelength velocity file paths and compute color range
velocity_shortwvl_filename = "velocity_shortwvl.h5"
fp_velocity_shortwvl, fp_comp_velocity_shortwvl = build_epoch_layer_paths(
    filename=velocity_shortwvl_filename,
    parent_dir=parent_dir,
    frame_num=frame_num,
    primary_version=version_num,
    comparison_version=version_num_to_comp,
)
ensure_file_exists(
    fp_velocity_shortwvl,
    f"{version_num} short wavelength velocity file",
)
if fp_comp_velocity_shortwvl:
    ensure_file_exists(
        fp_comp_velocity_shortwvl,
        f"{version_num_to_comp} short wavelength velocity file",
    )
(velocity_shortwvl_vmin, velocity_shortwvl_vmax) = compute_displacement_range(fp_velocity_shortwvl,
    dataset="velocity",)
velocity_shortwvl_vmin_comp = velocity_shortwvl_vmin
velocity_shortwvl_vmax_comp = velocity_shortwvl_vmax
if fp_comp_velocity_shortwvl and os.path.exists(fp_comp_velocity_shortwvl):
    (velocity_shortwvl_vmin_comp, velocity_shortwvl_vmax_comp) = compute_displacement_range(fp_comp_velocity_shortwvl,
    dataset="velocity",)
    print(f"Comparison velocity_shortwvl range: vmin={velocity_shortwvl_vmin_comp:.3f}, vmax={velocity_shortwvl_vmax_comp:.3f}")
print(
    f"Primary short wavelength velocity file: "
    f"{fp_velocity_shortwvl}"
)
print(
    "Short Wavelength Velocity range (cm/year): vmin="
    f"{velocity_shortwvl_vmin:.3f}, "
    f"vmax={velocity_shortwvl_vmax:.3f}"
)
if fp_comp_velocity_shortwvl:
    print(
        "Comparison short wavelength velocity file: "
        f"{fp_comp_velocity_shortwvl}"
    )
else:
    print(
        "Comparison version not provided; skipping comparison "
        "short wavelength velocity path."
    )




In [ ]:
display(
    Markdown(
        f"#### {version_num} Short Wavelength Velocity"
    )
)
velocity_shortwvl_view = (
    f"-v {velocity_shortwvl_vmin:.3f} {velocity_shortwvl_vmax:.3f} "
    "--noverbose --zm -u cm/year"
)
capture_view(fp_velocity_shortwvl, velocity_shortwvl_view)


In [ ]:
if version_num_to_comp and fp_comp_velocity_shortwvl:
    display(
        Markdown(
            f"#### {version_num_to_comp} Short Wavelength "
            "Velocity"
        )
    )
    velocity_shortwvl_comp_view = (
        f"-v {velocity_shortwvl_vmin:.3f} "
        f"{velocity_shortwvl_vmax:.3f} "
        "--noverbose --zm -u cm/year"
    )
    capture_view(
        fp_comp_velocity_shortwvl,
        velocity_shortwvl_comp_view,
    )
else:
    print(
        "No comparison version available for short wavelength "
        "velocity visualization."
    )


In [ ]:
# Get min/max of all datasets
report_combined_range(
    description="Short Wavelength Velocity",
    primary_path=fp_velocity_shortwvl,
    comparison_path=fp_comp_velocity_shortwvl,
    dataset="velocity",
    scale=100.0,
    primary_range=(velocity_shortwvl_vmin, velocity_shortwvl_vmax),
)


In [ ]:

if version_num_to_comp and fp_comp_velocity_shortwvl:
    diff_figures_dir = Path(ensure_directory(Path(figures_dir) / "differences"))
    diff_outname = diff_figures_dir / Path(fp_comp_velocity_shortwvl).name
    if not diff_outname.exists():
        scp_args = f"{fp_velocity_shortwvl} {fp_comp_velocity_shortwvl} -o {diff_outname}"
        diff.main(scp_args.split())
    display(Markdown("##### Short Wavelength Velocity Difference"))
    capture_view(str(diff_outname), velocity_shortwvl_view)
else:
    print(
        "Skipping short wavelength velocity difference; "
        "comparison file is unavailable."
    )



## 6. Range Summary

This section aggregates the reported ranges to make it easy to 
spot unexpected values across products.


In [ ]:

# ============================
# 6. Range Summary
# ============================

def _render_range_summary_table() -> None:
    if not _COMBINED_RANGE_SUMMARY:
        display(Markdown('No combined ranges were recorded in this run.'))
        return

    rows = []
    for entry in _COMBINED_RANGE_SUMMARY:
        label = entry['label']
        anchor = entry.get('anchor')
        target = f"<a href='{anchor}'>{label}</a>" if anchor else label
        rows.append(
            '<tr>'
            f"<td>{target}</td>"
            f"<td>{entry['units']}</td>"
            f"<td>{entry['min']:.3f}</td>"
            f"<td>{entry['max']:.3f}</td>"
            '</tr>'
        )

    header = (
        "<table style='border-collapse:collapse;'>"
        "<thead>"
        "<tr>"
        "<th style='padding:4px 8px;text-align:left;'>Section / Layer</th>"
        "<th style='padding:4px 8px;text-align:left;'>Units</th>"
        "<th style='padding:4px 8px;text-align:left;'>Min</th>"
        "<th style='padding:4px 8px;text-align:left;'>Max</th>"
        "</tr>"
        "</thead>"
        "<tbody>"
    )
    table_html = header + ''.join(rows) + '</tbody></table>'
    display(HTML(table_html))


_render_range_summary_table()
